# Introduction

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Understanding "HOW" to access LLMs

There are 3 main ways in which we can access LLMs, and each have their own pros and cons. 

1. Leading models are typically **closed-source/proprietary** and available through an API.

    - Some popular examples are OpenAI models (think GPT-4); Anthropic models (think Claude 3.7 Sonnet); Google models (think Gemini 2.0 Flash). 
    - This means that the data is sent to these LLM providers and can be used for further training. 
        - A workaround is to access these through cloud intermediaries such as Amazon Bedrock, Microsoft Azure or Google Vertex.
    - The costing is done at a token level, and no infrastructure needs to be set up. 


2. The main other bucket of models is **open-source**. There are 2 main ways in which we can access these.

    - Option 1 - Through API (larger models): 
        - Even though open-source, it's advised to use larger models, such as DeepSeek-V3-0324 (671B), through an API such as Groq. Groq makes the inference very fast!
        - The data is also shared with Groq in this case, so confidentiality isn't maintained. 
        - Costing is done at a token level, and no infrastructure needs to be set up.  
    
    - Option 2 - Locally host (smaller models):
        - This means that you are downloading the parameters of the LLM model on your system (it can be Google Colab / your cloud instance also), and then serving the model through libraries such as vLLM or Ollama. Think of models such as Llama 3.2 (3B).
            - vLLM is good for high-throughput scenarios with lots of users, whereas Ollama is more for day-to-day simplicity and usage. We'll use Ollama in our course. 
        - Data confidentiality is maintained, as there is no outside API call. 
        - Since you are hosting the model, costing is done based on the infrastructure usage (think GPU usage!).
    

## Which LLMs will we be exploring in this notebook

In [2]:
# LLMs vary in latency across different providers

In [3]:
# Different LLMs fit different use cases

We'll start with exploring the following models, and you're free to explore more models as you see fit:

*LARGER MODELS*

**Group 1. Closed-source (only through API; number of parameters not disclosed generally) - OPTIONAL IN THIS SECTION**:
- OpenAI API - Paid Tier only (min \$5 top-up, pay-per-use)
    - GPT 4.1-Mini
    - GPT 4o-Mini
- Anthropic API - Paid Tier only (min $5 top-up, pay-per-use)
    - Claude 3.7 Sonnet
- Google API - Free Tier Available
    - Gemini 2.0 Flash    

**Group 2. Open-source (Option 1 - through API)**:
- Groq API (all are open-source) - Free Tier Available
    - Llama 3.3 70B (70B)
    - DeepSeek-R1-Distill-Llama-70B (70B)
- DeepSeek API (some are open-source) - Paid Tier only (min $2 top-up, pay-per-use)
    - DeepSeek-R1-0528 (671B)

*SMALLER MODELS*

**Group 3. Open-source (Option 2 - hosting locally with Ollama)**:
- Very small, very fast inference
    - Llama 3.2 (1B) - 1.3GB
    - DeepSeek-R1-Distill-Qwen-1.5B (1.5B) - 1.1GB
- Small size, fast inference
    - Llama 3.2 (3B) - 2GB
    - Phi 3 (3.8B) - 2.2GB
    - Gemma 3 (4B) - 3.3GB
- Slightly larger size, slightly slower inference
    - Qwen 3 (8B) - 5.2GB
    - Llama 3.1 (8B) - 4.9GB
    - DeepSeek-R1-Distill-Qwen-7B (7B) - 4.7GB

# Getting Started

## Setup Issues and Resolutions

Just to check you've already added the Python and Jupyter extensions to Cursor, if not already installed:
- Open extensions (View >> extensions)
- Search for python, and when the results show, click on the ms-python one, and Install it if not already installed
- Search for jupyter, and when the results show, click on the Microsoft one, and Install it if not already installed  
Then View >> Explorer to bring back the File Explorer.

And then:
1. Click where it says "Select Kernel" near the top right, and select the option called `.venv (Python 3.12.9)` or similar, which should be the first choice or the most prominent choice. You may need to choose "Python Environments" first.
2. Click in each "cell" below, starting with the cell immediately below this text, and press Shift+Enter to run
3. Enjoy!

After you click "Select Kernel", if there is no option like `.venv (Python 3.12.9)` then please do the following:  
1. On Mac: From the Cursor menu, choose Settings >> VS Code Settings (NOTE: be sure to select `VSCode Settings` not `Cursor Settings`);  
On Windows PC: From the File menu, choose Preferences >> VS Code Settings(NOTE: be sure to select `VSCode Settings` not `Cursor Settings`)  
2. In the Settings search bar, type "venv"  
3. In the field "Path to folder with a list of Virtual Environments" put the path to the project root, like C:\Users\username\projects\eo-dev-onboarding (on a Windows PC) or /Users/username/projects/eo-dev-onboarding (on Mac or Linux).  
And then try again.

Having problems with missing Python versions in that list? Have you ever used Anaconda before? It might be interferring. Quit Cursor, bring up a new command line, and make sure that your Anaconda environment is deactivated:    
`conda deactivate`  
And if you still have any problems with conda and python versions, it's possible that you will need to run this too:  
`conda config --set auto_activate_base false`  
and then from within the **eo-dev-onboarding** directory, you should be able to run `uv python list` and see the Python 3.12 version.

## Import Libraries

In [5]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [6]:
# Always remember to do this! - What does this do?
load_dotenv(override=True)

True

# Concept: OpenAI Compatible Endpoints

An **industry standard**, compatibility with OpenAI's client library is applicable to **most closed-source models** as we'll see, other APIs such as **Groq and DeepSeek**, as well as **Ollama** that allows us to use open-source models.

## Step 0 - Setting up model

In [7]:
#Set API Key for LLM (Groups 1 and 2 only) - example for OpenAI API here
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key exists and begins sk-proj-


In [8]:
# Create an instance of the OpenAI class - for both open-source and closed-source, with slight variations
llm_api = OpenAI()

In [9]:
# Select model 
model_name = "gpt-4.1-nano"

## Step 1 - Setting up input to LLM

In [10]:
# Raw user prompt
request = "Please come up with a challenging, nuanced question in Machine Learning and AI that I can ask a number of LLMs to evaluate their intelligence. "
request += "Some examples of questions that I have asked in the past are: "
request += "1. What is conformal prediction and how does it work? Explain with an example. "
request += "2. What is McFadden's pseudo R-squared? Explain with an example. "
request += "3. What is Platt scaling for multi-class classification? Explain with an example."
request += "Answer only with the question, no explanation."

In [11]:
# Create a list of messages in the familiar OpenAI format
messages = [{"role": "user", "content": request}]

## Step 2

OpenAI Official documentation is here - https://platform.openai.com/docs/api-reference/chat/create Refer to this for a full set of parameters that we'll later explore.

In [12]:
# Call the Chat Completions endpoint
response = llm_api.chat.completions.create(
    model=model_name,
    messages=messages,
    seed =42
)

## Step 3

In [13]:
# See raw response
print(response)

ChatCompletion(id='chatcmpl-CE9fV1S20yab7NsBd86mj147M9Qbg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='How can we rigorously evaluate and compare the uncertainty quantification capabilities of different probabilistic models—such as Bayesian neural networks, ensemble methods, and conformal prediction—especially in high-stakes, real-world applications where ensuring calibrated confidence estimates is critical?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1757489793, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_7c233bf9d1', usage=CompletionUsage(completion_tokens=50, prompt_tokens=114, total_tokens=164, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cac

In [14]:
# See only output
question = response.choices[0].message.content
print(question)

How can we rigorously evaluate and compare the uncertainty quantification capabilities of different probabilistic models—such as Bayesian neural networks, ensemble methods, and conformal prediction—especially in high-stakes, real-world applications where ensuring calibrated confidence estimates is critical?


# Performing Same Thing with TFY AI Gateway

In [15]:
#Need to pass API Key as an argument for all LLMs
llm_api_tfy = OpenAI(api_key=os.environ["TFY_API_KEY"], base_url="https://internal.devtest.truefoundry.tech/api/llm")

In [16]:
# Select model 
model_name = "groq-test-anant/openai-gpt-oss-120b"

**Note that I have already added the models in the AI Gateway - so it's ready to use, but that's a one-time effort typically.**

## Step 1 - Setting up input to LLM

In [17]:
# Raw user prompt
request = "Please come up with a challenging, nuanced question in Machine Learning and AI that I can ask a number of LLMs to evaluate their intelligence. "
request += "Some examples of questions that I have asked in the past are: "
request += "1. What is conformal prediction and how does it work? Explain with an example. "
request += "2. What is McFadden's pseudo R-squared? Explain with an example. "
request += "3. What is Platt scaling for multi-class classification? Explain with an example."
request += "Answer only with the question, no explanation."

In [18]:
# Create a list of messages in the familiar OpenAI format
messages = [{"role": "user", "content": request}]

## Step 2

OpenAI Official documentation is here - https://platform.openai.com/docs/api-reference/chat/create Refer to this for a full set of parameters that we'll later explore.

In [19]:
# Call the Chat Completions endpoint
response = llm_api_tfy.chat.completions.create(
    model=model_name,
    messages=messages,
    seed =42
)

## Step 3

In [20]:
# See raw response
print(response)

ChatCompletion(id='chatcmpl-24f5970b-78fd-44d0-852a-284943cf97a0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Explain the concept of “counterfactual regret minimization” (CFR) as used in game‑theoretic reinforcement learning for solving extensive‑form games, describe how the algorithm constructs and updates regret tables across information sets, and illustrate with a concrete example (e.g., a simplified poker hand) how CFR converges to a Nash equilibrium, including a discussion of any assumptions or limitations of the method.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning='The user wants a challenging, nuanced question in ML/AI to ask LLMs to evaluate their intelligence. They want only the question, no explanation. So we need to output a single question. Should be challenging and nuanced. Could involve advanced topics like causal inference, Bayesian deep learning, n

In [21]:
# See only output
question = response.choices[0].message.content
print(question)

Explain the concept of “counterfactual regret minimization” (CFR) as used in game‑theoretic reinforcement learning for solving extensive‑form games, describe how the algorithm constructs and updates regret tables across information sets, and illustrate with a concrete example (e.g., a simplified poker hand) how CFR converges to a Nash equilibrium, including a discussion of any assumptions or limitations of the method.


# What It Takes to Use Different LLMs (without AI Gateway)

## Getting API Keys Ready

In [22]:
# Print the key prefixes to help with any debugging
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_


## Initializing Task (comparing performance across LLMs)

In [23]:
# Wrapping Steps 1-3 in a function
def get_output(llm_api, model_name, messages):
    response = llm_api.chat.completions.create(
        model=model_name,
        messages=messages
)
    return(response.choices[0].message.content)

In [24]:
# question = "question ......"

In [25]:
competitor_llms = [] #list of LLMs that'll be called
answers = [] #answers from each LLM
messages = [{"role": "user", "content": question}] #messages

## Groups 1 LLMs - Closed Source

### OpenAI API

In [26]:
# GPT 4.1-Mini
llm_api = OpenAI() #native to OpenAI models
model_name = "gpt-4.1-mini"

answer = get_output(llm_api, model_name, messages)

competitor_llms.append(model_name)
answers.append(answer)

display(Markdown(answer))

Certainly! Below is a detailed explanation of **Counterfactual Regret Minimization (CFR)** as used in game-theoretic reinforcement learning, specifically for solving **extensive-form games** such as poker.

---

### Overview: What is CFR?

**Counterfactual Regret Minimization (CFR)** is an iterative algorithm designed to find approximate **Nash equilibria** in extensive-form games (games with sequential moves, imperfect information, and chance events). It was introduced by Zinkevich et al. (2007) and has been very influential in computational poker and other domains involving imperfect information.

**Key idea:**  
CFR minimizes **counterfactual regret** for each player over possible actions at each information set. Over many iterations, the average strategy converges to a Nash equilibrium strategy.

---

### Key Concepts

1. **Extensive-form games**:

   - Represented as a game tree.
   - Players do not have perfect information—nodes are grouped into **information sets**, which represent what a player knows when making a decision.
   - Players pick strategies mapping information sets to action probabilities.

2. **Regret in decision-making:**

   - **Regret** is the difference in utility between the action taken and the best action in hindsight.
   - Minimizing regret over time is related to approaching an equilibrium.

3. **Counterfactual values:**

   - The value of an action assuming the player **had reached the information set** but **counterfactually** consider what would happen if a different action were taken.
   - Helps compute regret locally without generating the entire game tree.

---

### How CFR Works: Algorithmic Description

#### Notation:

- Let \( I \) be an **information set** for player \( i \).
- Let \( A(I) \) be the set of actions available at \( I \).
- Let \( \sigma^t \) be the strategy profile at iteration \( t \).
- \( r^t(I, a) \) = **regret** for not having played action \( a \) at \( I \) at iteration \( t \).
- \( R^T(I, a) = \sum_{t=1}^T r^t(I, a) \) = **cumulative regret** over \( T \) iterations.

---

#### Step 1: Initialize Regret Tables

- For each information set \( I \), initialize cumulative regrets \( R(I, a) = 0 \) for all \( a \in A(I) \).
- Initialize strategy tables to some initial strategy (often uniform random).

---

#### Step 2: Iterative Simulation

At each iteration \( t \):

1. **Traverse the game tree** via **recursive calls**, calculating "counterfactual utilities":
   - For each information set \( I \), compute the counterfactual value of each possible action \( a \):
     \[
     v^{\sigma}_{i}(I, a) = \text{expected payoff if player } i \text{ plays } a \text{ at } I \text{ and follows } \sigma \text{ elsewhere}
     \]
   - Compute the overall counterfactual value:
     \[
     v^{\sigma}_{i}(I) = \sum_{a \in A(I)} \sigma(I, a) \cdot v^{\sigma}_{i}(I, a)
     \]
   
2. **Calculate the instantaneous regret for each action:**
   \[
   r^t(I, a) = v^{\sigma}_{i}(I, a) - v^{\sigma}_{i}(I)
   \]

3. **Update cumulative regret:**
   \[
   R^{t}(I, a) = R^{t-1}(I, a) + r^{t}(I, a)
   \]

4. **Update strategy for next iteration using regret-matching:**

   Define the positive regrets as:
   \[
   R^{+}(I, a) = \max(R^t(I, a), 0)
   \]
   
   Then:
   \[
   \sigma^{t+1}(I, a) = \frac{R^{+}(I, a)}{\sum_{a' \in A(I)} R^{+}(I, a')} \quad \text{if denominator} > 0,
   \]
   otherwise pick uniform strategy.

---

#### Step 3: Average the strategies

- The **average strategy** across iterations \( \bar{\sigma} \) is what converges to a Nash equilibrium, not individual iterates \( \sigma^t \).
- Formally:
  \[
  \bar{\sigma}(I, a) = \frac{\sum_{t=1}^T \pi_i^{\sigma^t}(I) \sigma^t(I, a)}{\sum_{t=1}^T \pi_i^{\sigma^t}(I)}
  \]
  where \( \pi_i^{\sigma^t}(I) \) is the probability of reaching \( I \) under \( \sigma^t \).

---

### CFR on a Simplified Poker Example

---

#### Example: Simplified Heads-up Kuhn Poker

- **Setup:**
  - Deck: 3 cards {J, Q, K}.
  - Each player antes 1 chip.
  - Each player is dealt one card.
  - Player 1 can bet or check.
  - Player 2 can fold or call after a bet.
  - Payoffs depend on winning card and bets.
  
---

#### Steps:

1. **Information Sets:**

- Player 1's decision with J, Q, or K (not knowing Player 2's card).
- Player 2's decision depends on Player 1's action and own card.

2. **Regret Tables:**

At information set representing Player 1 having card \( c \) and acting first, actions are **bet** or **check**.

- Initialize \( R = 0 \) for bet and check.

Similarly for Player 2's info sets, with actions fold or call.

3. **Iterations:**

- Start with uniform strategies \( \sigma^1 \).
- Compute counterfactual values \( v \): expected values if players deviate.
- Calculate regret and update \( R \).
- Use regret matching to update \( \sigma^{t+1} \).
- Repeat for thousands of iterations.

---

#### CFR's properties in this context:

- Over time, average strategies approximate the Nash equilibrium strategy for Kuhn poker.
- For example, Player 1 learns to bet more often with K and less with J, Player 2 learns optimal call/fold thresholds.

---

### Assumptions and Limitations of CFR

---

#### Assumptions:

- **Perfect recall:** Players remember their own moves and knowledge.
- The game must be **finite** with a manageable representation of the game tree or approximate abstraction.
- Rewards/payoffs are fixed and known.

---

#### Limitations:

- **Scalability:** The full game tree for large games (like real Texas Hold’em) is enormous.
- Requires **abstractions:** Grouping similar states to reduce complexity, which may cause approximation errors.
- **Convergence speed:** Though theoretically proven to converge at a \( O(1/\sqrt{T}) \) rate, practical convergence may need very many iterations.
- Extensions like **Monte Carlo CFR** have been developed to sample game paths to reduce computational burden.
- CFR finds **Nash equilibria** but not necessarily equilibria refined by other criteria.

---

### Summary

| Step in CFR        | Explanation                                                  |
|--------------------|--------------------------------------------------------------|
| Initialize regrets  | Start with zero regret for all actions in all info sets.     |
| Traverse game tree  | Compute counterfactual values for actions at each info set.  |
| Compute regrets     | Difference between value of each action & current strategy.  |
| Update regrets      | Accumulate regret for each action over iterations.           |
| Update strategies   | Use regret matching to pick new strategies favoring positive regrets. |
| Average strategies  | Average over iterations converges to Nash equilibrium.       |

---

### References for further reading

- Zinkevich, M., Johanson, M., Bowling, M., & Piccione, C. (2007). *Regret Minimization in Games with Incomplete Information*. NIPS 2007.
- Bowling, M., et al. (2015). *Heads-up Limit Hold’em Poker is Solved*. Science.
- Brown, N., & Sandholm, T. (2019). *Superhuman AI for multiplayer poker*. Science.

---

I hope this helps clarify how CFR works in extensive-form games, how it uses regret tables at information sets to iteratively improve strategies, and how it converges to Nash equilibria using the example of simplified poker! Let me know if you want code snippets or more mathematical details.

In [27]:
# GPT 4o-Mini
llm_api = OpenAI() #native to OpenAI models
model_name = "gpt-4o-mini"

answer = get_output(llm_api, model_name, messages)

competitor_llms.append(model_name)
answers.append(answer)

display(Markdown(answer))

### Counterfactual Regret Minimization (CFR)

Counterfactual Regret Minimization (CFR) is a central concept in solving extensive-form games, particularly in the realm of game-theoretic reinforcement learning. It is designed to help agents find a Nash equilibrium by iteratively minimizing the regret of their strategies through repeated simulations of the game.

#### Key Concepts

1. **Extensive-form Games**: These are games represented as trees where nodes represent decision points and edges represent actions taken by players. Players can have imperfect information about other players' actions.

2. **Regret**: The regret for a particular strategy is the difference between the payoff obtained using this strategy and the payoff that could have been obtained if the player had used a different action in the same situation.

3. **CFR Iteration**: The algorithm uses a repeated simulation approach, calculating the regret at each iteration and adjusting strategies accordingly. This process allows players to refine their strategies based on past experiences.

### Regret Tables and Information Sets

CFR maintains what's called a "regret table" for each information set of a player, which records the regrets for each action. Here’s how the algorithm constructs and updates these tables:

1. **Initialization**: Start with zero regret for all actions available at each information set.

2. **Sampling**: Play out the game using a strategy derived from the regret table. The strategies are usually a mixture based on current regrets (e.g., using a softmax or a regret-matching strategy).

3. **Regret Calculation**:
   - During each play, for each information set, track the counterfactual values: what a player would have received (the payoff) if they had chosen each action instead of the one they actually took.
   - Update the regret table for each action based on the payoffs. The regret for taking an action \(a\) in information set \(I\) is updated as:
     \[
     R_I(a) = R_I(a) + \text{counterfactual payoff} - \text{actual payoff}
     \]

4. **Strategy Update**: After several iterations, refine the strategy based on accumulated regrets, often using regret-matching:
   - If \(R_I(a) > 0\), then increase the probability of choosing action \(a\) according to its regret relative to the total positive regret for that information set.
   - Normalize the probabilities across actions so that they sum to one.

### Example: Simplified Poker Hand

Imagine a simplified poker game with two players, where each player can either bet or fold. The game unfolds as follows:

- **Game Initialization**: Each player starts with a certain amount of chips.
- **Decision Node**: Player A can either Bet or Fold. If Player A Bets, Player B can either Call or Fold.

#### CFR Process
1. **Iterate Plays**: Begin with a uniform strategy (both players randomly select actions).
2. **Play the Game**: Play the game many times, each time recording the outcomes.
   - For example, suppose Player A Bets (which gets a payoff of +5 for A if B folds, and -5 if B calls but loses).
   - Record payoffs (actual) and update each player’s regrets based on what could have happened if they had acted differently.
3. **Regret Accumulation**: After many iterations, the regrets will reflect:
   - If Player A consistently gains more by betting rather than folding (they accumulate a positive regret for folding), and vice versa for Player B.

4. **Strategy Convergence**: Over time, the players adjust their strategies and the regrets stabilize, moving towards a Nash equilibrium where neither player has an incentive to deviate from their strategy.

### Assumptions and Limitations

1. **Perfect Sampling**: CFR assumes that all possible plays are sampled adequately, which can be challenging in large games due to exponential growth in game tree size.

2. **Sufficient Iterations**: Convergence can take a large number of iterations, and the quality of the equilibrium improves with more thorough exploration.

3. **Non-Exponential Information Sets**: CFR may struggle with very large information sets or highly complex strategies where maintaining and updating a regret table becomes computationally infeasible.

4. **Strategies with Noise**: Players might take actions that are explorative, meaning probabilistic strategies characterized by exploration versus exploitation must be maintained effectively to ensure convergence.

In summary, CFR is powerful for analyzing strategic decision-making in games and converging to Nash equilibria through systematic regret minimization across various strategies. However, it does face practical challenges in scalability and computational resources when applied to highly complex situations.

### Anthropic API

**Note that Anthropic has a slightly different API, and max tokens is also required (not OpenAI compatible!)**

In [28]:
# Claude 3.7 Sonnet
llm_api = Anthropic()
model_name = "claude-3-7-sonnet-latest"

response = llm_api.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

competitor_llms.append(model_name)
answers.append(answer)

display(Markdown(answer))

# Counterfactual Regret Minimization in Game Theory

## Core Concept

Counterfactual Regret Minimization (CFR) is an iterative algorithm for approximating Nash equilibria in extensive-form games, particularly those with imperfect information. The fundamental insight behind CFR is to minimize "regret" at each decision point by adjusting strategy profiles based on how much better a player could have done by choosing differently in past iterations.

## Key Elements of CFR

### 1. Information Sets

In imperfect information games, players cannot distinguish between certain game states. These indistinguishable states are grouped into **information sets**. For example, in poker, a player's information set includes all possible private card combinations their opponent might have.

### 2. Regret and Counterfactual Value

- **Regret**: The difference between the utility a player would have received by always choosing a particular action and the utility they actually received.
- **Counterfactual value**: The expected utility of taking an action, weighted by the probability of reaching the information set where that action is taken (assuming the player tries to reach it).

### 3. Algorithm Structure

CFR works by:
1. Calculating counterfactual values for each action at each information set
2. Computing regret for not taking each action
3. Accumulating these regrets over iterations
4. Defining a new strategy based on positive accumulated regrets
5. Repeating until convergence

## Regret Tables and Updates

For each information set $I$ and action $a$, CFR maintains:

1. **Cumulative regret** $R^T(I,a)$: The sum of regrets for not choosing action $a$ at information set $I$ over T iterations
2. **Strategy profile** $\sigma^T(I,a)$: The probability of choosing action $a$ at information set $I$ at iteration T
3. **Average strategy profile** $\bar{\sigma}^T(I,a)$: The average of all strategy profiles up to iteration T

### Update Process

1. At each iteration $t$:
   - Calculate counterfactual values $v(I,a)$ for each action
   - Calculate immediate regret: $r^t(I,a) = v(I,a) - v(I)$
   - Update cumulative regret: $R^{t+1}(I,a) = R^t(I,a) + r^t(I,a)$

2. Calculate the next strategy using regret matching:
   $\sigma^{t+1}(I,a) = \frac{R^{t+1}_+(I,a)}{\sum_{a'} R^{t+1}_+(I,a')}$
   
   where $R^{t+1}_+(I,a) = \max(R^{t+1}(I,a), 0)$

3. Update the average strategy, which converges to a Nash equilibrium

## Concrete Example: Simplified Poker Hand

Let's consider a simplified poker game:
- Two players: P1 and P2
- P1 is dealt either a High (H) or Low (L) card with equal probability
- P1 can either Bet (B) or Check (C)
- If P1 Bets, P2 can Call (C) or Fold (F)
- If P1 Checks, the game ends with a showdown

### Game Tree:
```
          P1
         /  \
        /    \
     H(0.5)  L(0.5)
      / \     / \
     B   C   B   C
    / \     / \
P2 C   F   C   F
   |   |   |   |
  +1  -1  -1  +1  (Payoffs for P1)
```

### CFR Execution

**Iteration 1:**
- Initial strategy: Uniform random (0.5 for each action)
- Information sets for P1: {H}, {L}
- Information set for P2: {P1 bet} (P2 doesn't know P1's card)

Let's calculate regrets for P1 with card H:
- EV(Bet) = 0.5 

### Google API

In [29]:
# Gemini 2.0 Flash
llm_api = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

answer = get_output(llm_api, model_name, messages)

competitor_llms.append(model_name)
answers.append(answer)

display(Markdown(answer))

## Counterfactual Regret Minimization (CFR) Explained

Counterfactual Regret Minimization (CFR) is a powerful iterative algorithm used to approximate Nash equilibria in large, imperfect-information games, such as poker. It is a core technique in game-theoretic reinforcement learning. The central idea behind CFR is to learn a policy that minimizes the **counterfactual regret**, which quantifies how much better the agent would have performed if it had chosen a different action in a particular information set, assuming all other players played the same way.

**Key Concepts:**

* **Extensive-Form Games:**  Games represented as a tree structure, capturing the sequence of actions, player turns, private information (e.g., hidden cards), and chance outcomes. Examples include poker, chess, and negotiation scenarios.
* **Information Set:** A set of game states (nodes in the game tree) that a player cannot distinguish between. The player has the same knowledge and available actions at all nodes within the same information set. For example, in Texas Hold'em, a player's information set includes all board states and hole cards that are consistent with the player's own holdings and the actions taken so far.
* **Policy:** A probability distribution over actions for each information set. It dictates how a player will choose their next action given the current state of knowledge.
* **Nash Equilibrium:** A set of strategies (one for each player) where no player can improve their expected payoff by unilaterally changing their strategy, assuming all other players' strategies remain fixed. It represents a stable state in the game.
* **Counterfactual Value:** The expected value (payoff) of reaching a specific information set and following a particular action, assuming all other players play their current strategies and the agent plays optimally from that point onward. It's "counterfactual" because it considers what *would have happened* if the agent had reached that information set.
* **Regret:** A measure of how much better the agent *would have done* if it had consistently chosen a specific action in a specific information set, given what actually happened in past iterations.
* **Counterfactual Regret:**  The regret associated with an action, weighted by the probability of reaching the information set. Specifically, it's the difference between the counterfactual value of taking a specific action and the counterfactual value of following the current policy in that information set.
* **Regret Matching:** A strategy update rule that chooses actions proportionally to their positive regrets.  Actions with higher positive regret are played more often in future iterations.

**How CFR Works: Building and Updating Regret Tables**

CFR iteratively traverses the game tree, performing the following steps for a large number of iterations:

1. **Tree Traversal:** The algorithm starts at the root node of the game tree and recursively explores the game, simulating play according to the current policies of all players.
2. **Value Evaluation (Backpropagation):** As the game tree is explored, the algorithm calculates and propagates values upward. These values are used to estimate the counterfactual values of each information set and action.
3. **Regret Calculation:**  For each information set the agent encounters during the traversal, the algorithm calculates the counterfactual regret for each possible action. This is done by:
   * **Calculating the counterfactual value of each action:**  Estimating the expected payoff if the agent always chose this action from this information set onwards, while other players continue to play according to their current strategies.
   * **Subtracting the expected value of the current policy from each action's counterfactual value:** This difference is the regret for choosing that action.
4. **Regret Accumulation:** The calculated regrets are added to a regret table associated with each information set.  The regret table is a dictionary where keys are actions and values are the cumulative regret for each action.
5. **Policy Update (Regret Matching):** After traversing the entire game tree, the algorithm updates the agent's policy for each information set based on the accumulated regrets.  It uses a **regret matching** algorithm.  This means that the probability of choosing an action is proportional to the positive accumulated regret for that action. Actions with high regret (meaning the agent would have done better choosing them in the past) are played more frequently in future iterations.  Negative regrets are typically truncated to zero.
6. **Iteration:** Repeat steps 1-5 for many iterations.  Over time, the agent's policy converges towards a Nash equilibrium.

**Mathematical Representation:**

Let:

* `I` be an information set.
* `A(I)` be the set of actions available in information set `I`.
* `π(a)` be the probability of taking action `a`.
* `u(I, a)` be the counterfactual value of taking action `a` in information set `I`.
* `π(I)` be the probability of reaching information set `I`.
* `r_t(I, a)` be the regret for taking action `a` in information set `I` at time `t`.
* `R_T(I, a)` be the cumulative regret for action `a` in information set `I` after `T` iterations.

The regret update rule is:

`R_{T+1}(I, a) = R_T(I, a) + π(I) * (u(I, a) - ∑_{a' ∈ A(I)} π(a') * u(I, a'))`

The policy update rule using regret matching is:

`π_{T+1}(a) = R_T^+(I, a) / ∑_{a' ∈ A(I)} R_T^+(I, a')`  where `R_T^+(I, a) = max(R_T(I, a), 0)` (truncate negative regret to zero).

**Concrete Example: Simplified Poker Hand (Kuhn Poker)**

Kuhn Poker is a simplified two-player poker variant with only three cards (Jack, Queen, King). Each player antes one chip, and each player is dealt one card. Player 1 acts first. They can either bet or pass. If Player 1 bets, Player 2 can either call or fold. If Player 1 passes, Player 2 can either bet or pass. If both players pass, or if Player 1 bets and Player 2 calls, the player with the higher card wins the pot (2 chips). If a player folds, the other player wins the pot.

Let's see how CFR would converge to a Nash equilibrium in this game (very simplified illustration, focusing on policy update).

**Iteration 1:**

* **Initial Policies:** Assume both players start with uniform random policies (33.3% probability of betting/passing/calling/folding, as applicable to their current situation).

* **Traversal & Value Evaluation:**  The algorithm traverses the entire game tree, simulating games using these initial policies and calculating counterfactual values.  This involves calculating the expected value for each action in each information set.

* **Regret Calculation:**  Let's say Player 1 holds a Queen and Player 2's initial policy is to call 50% of the time when Player 1 bets. Suppose, after value evaluation, Player 1 calculates that betting with a Queen has a counterfactual value of 0.2 (meaning they win a small amount on average).  Passing with the Queen has a counterfactual value of -0.1 (they lose a small amount on average).

* **Regret Update:** Player 1's regrets for the actions in the information set "holding a Queen, Player 1 acts first" would be:

    * Regret(Bet):  0.2 - (0.33 * 0.2 + 0.33 * -0.1 + 0.33 * ... other actions) = some positive value.  Let's say it is 0.2
    * Regret(Pass): -0.1 - (0.33 * 0.2 + 0.33 * -0.1 + 0.33 * ... other actions) = some negative value.  Let's say it is -0.1

* **Policy Update:**  Applying regret matching (truncating negative regret):

    * Total Positive Regret = max(0.2, 0) + max(-0.1, 0) = 0.2
    * P(Bet) = 0.2 / 0.2 = 1
    * P(Pass) = 0 / 0.2 = 0

    So, in the next iteration, Player 1 will *always* bet when holding a Queen and acting first.

**Iteration 2 and Beyond:**

This process is repeated for many iterations. Players adjust their policies based on accumulated regrets. For example, in Kuhn poker:

* **Bluffing:** Over time, Player 1 will learn to bet with the Jack some fraction of the time (bluffing) to make their betting range less predictable.
* **Value Betting:** They'll learn to bet with the King more frequently.
* **Calling/Folding:** Player 2 will learn optimal calling/folding frequencies based on the bets they face.

**Convergence:**  As the number of iterations increases, the policies converge towards a Nash equilibrium. In Kuhn Poker, the Nash equilibrium is a mixed strategy, involving betting, passing, calling, and folding with certain probabilities.  CFR, through its regret minimization and policy updating, will approximate these probabilities.

**Assumptions and Limitations of CFR:**

* **Perfect Recall:** CFR assumes that players have perfect recall, meaning they remember all their previous actions and observations.  This may not be true for humans, especially in complex games.
* **Computational Cost:** CFR can be computationally expensive, especially for large games with deep game trees. The memory requirements for storing regret tables can also be substantial.
* **Approximate Equilibrium:** CFR doesn't guarantee finding the *exact* Nash equilibrium, but it provides a good approximation. The approximation quality depends on the number of iterations.
* **Adversary Independence:** CFR assumes that opponents are playing fixed strategies. It doesn't explicitly model opponent adaptation. However, the iterative nature of CFR means it implicitly adapts to the average strategy played by opponents over many iterations.
* **Static Game Structure:** CFR assumes a static game structure. If the rules of the game change, the algorithm needs to be re-run.
* **Scalability Issues:** For extremely large games, standard CFR can become intractable. Variants like Monte Carlo CFR (MCCFR) use sampling techniques to explore only a portion of the game tree, significantly reducing computational costs but potentially increasing variance.

**In Summary:**

CFR is a powerful algorithm for approximating Nash equilibria in imperfect-information games. By iteratively calculating and minimizing counterfactual regret, it allows players to learn optimal strategies through self-play. While it has limitations in terms of computational cost, assumptions about perfect recall, and adversary independence, it has been successfully applied to a range of challenging games, including poker, demonstrating its effectiveness in complex strategic environments.  Modern research focuses on scaling CFR to even larger games and addressing its limitations, such as by incorporating opponent modeling or using more efficient sampling techniques.


## Groups 2 LLMs - Open Source via API

### Groq API

In [30]:
# Llama 3.3 70B through Groq - note that Grok is different and it's an LLM by X (formerly Twitter)
llm_api = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

answer = get_output(llm_api, model_name, messages)

competitor_llms.append(model_name)
answers.append(answer)

display(Markdown(answer))

Counterfactual Regret Minimization (CFR) is a game-theoretic reinforcement learning algorithm used to solve extensive-form games, which are games with imperfect information and multiple stages of play. CFR is designed to find a Nash equilibrium, a state in which no player can improve their payoff by unilaterally changing their strategy, assuming all other players keep their strategies unchanged.

**Key Components of CFR:**

1. **Regret Tables:** CFR maintains a regret table for each information set, which represents the cumulative regret of not playing a particular action in that information set. Regret is a measure of how much a player would have gained by playing a different action.
2. **Counterfactual Regret:** The counterfactual regret is the regret of not playing a particular action, assuming all other players play according to their current strategies.
3. **Strategy Updates:** CFR updates the strategy at each information set based on the regret table, using a regret-matching approach.

**Algorithmic Steps:**

1. **Initialization:** Initialize the regret tables and strategies for each information set.
2. **Simulation:** Simulate a trajectory of the game, using the current strategies to select actions.
3. **Regret Calculation:** Calculate the counterfactual regret for each action at each information set, considering the trajectory and the current strategies.
4. **Regret Table Update:** Update the regret table for each information set based on the calculated counterfactual regret.
5. **Strategy Update:** Update the strategy at each information set using a regret-matching approach, which selects actions with higher regret more frequently.
6. **Iteration:** Repeat steps 2-5 for a large number of iterations.

**Example: Simplified Poker Hand**

Consider a simplified poker game with two players, Alice and Bob. The game consists of two rounds of betting, with a single card dealt to each player. The cards are either high (H) or low (L). The payoffs are:

| Alice's Card | Bob's Card | Alice's Action | Payoff |
| --- | --- | --- | --- |
| H | H | Bet | +2 |
| H | L | Bet | +1 |
| L | H | Bet | -1 |
| L | L | Bet | -2 |
| H | H | Check | 0 |
| H | L | Check | 0 |
| L | H | Check | 0 |
| L | L | Check | 0 |

Initially, Alice's strategy is to bet with probability 0.5 and check with probability 0.5, regardless of her card. Bob's strategy is to call with probability 0.5 and fold with probability 0.5, regardless of his card.

**Convergence to Nash Equilibrium**

As CFR iterates, the regret tables and strategies are updated. Initially, Alice's regret table for the information set "H" might look like this:

| Action | Regret |
| --- | --- |
| Bet | 0.5 |
| Check | 0.5 |

After simulating many trajectories, the regret table might be updated to:

| Action | Regret |
| --- | --- |
| Bet | 0.8 |
| Check | 0.2 |

This indicates that Alice regrets not betting more often when she has a high card. The strategy update will increase the probability of betting when Alice has a high card.

Similarly, Bob's regret table for the information set "L" might be updated to:

| Action | Regret |
| --- | --- |
| Call | 0.4 |
| Fold | 0.6 |

This indicates that Bob regrets not folding more often when he has a low card. The strategy update will increase the probability of folding when Bob has a low card.

As CFR continues to iterate, the strategies converge to a Nash equilibrium, where neither player can improve their payoff by unilaterally changing their strategy. In this example, the Nash equilibrium might be:

* Alice bets with probability 0.8 when she has a high card and checks with probability 0.2.
* Bob calls with probability 0.4 when he has a low card and folds with probability 0.6.

**Assumptions and Limitations:**

1. **Complete Information:** CFR assumes that the game has complete information about the payoffs and the structure of the game.
2. **Finite Action Space:** CFR is designed for games with a finite action space.
3. **Zero-Sum Games:** CFR is primarily designed for zero-sum games, where one player's gain is equal to the other player's loss.
4. **Computational Complexity:** CFR can be computationally expensive, especially for large games.
5. **Convergence Rate:** The convergence rate of CFR can be slow, especially for games with many information sets.

In summary, Counterfactual Regret Minimization is a powerful algorithm for solving extensive-form games, but it requires careful consideration of the game's structure, payoffs, and assumptions. While CFR can converge to a Nash equilibrium, its limitations and assumptions must be carefully evaluated for each specific application.

In [31]:
# DeepSeek-R1-Distill-Llama-70B through Groq - note that Grok is different and it's an LLM by X (formerly Twitter)
llm_api = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "deepseek-r1-distill-llama-70b"

answer = get_output(llm_api, model_name, messages)

competitor_llms.append(model_name)
answers.append(answer)

display(Markdown(answer))

<think>
Okay, so I need to explain counterfactual regret minimization (CFR) as it's used in game-theoretic reinforcement learning for solving extensive-form games. I remember that CFR is an algorithm used in game theory, especially in games like poker where the game tree is huge. But I'm a bit fuzzy on the details, so I need to break it down.

First, I think CFR is about minimizing regret. Regret in this context is like saying, "If I had played a different strategy, would I have done better?" So, the algorithm tries to minimize this potential regret over time. That makes sense because in games, especially zero-sum games, you want to optimize your strategy to maximize your minimum gain.

CFR operates on extensive-form games, which are games that can be represented as a tree where each node is a decision point. Each information set groups nodes where the player can't distinguish between them based on the history of the game. So, for each information set, the algorithm maintains a regret table. I think the regret table keeps track of the regret for each possible action at that information set.

The algorithm works iteratively. In each iteration, it traverses the game tree and for each information set, it calculates the strategy or the probabilities of taking each action based on the current regret values. Then, it computes the counterfactual regret for each action, which is the difference in utility had that action been taken instead of the current strategy. These regrets are then updated in the table, and the process repeats.

I remember something about counterfactual values. Counterfactual regret is computed using these values, which are the expected utilities of taking a particular action at an information set, assuming that the opponent is following their current strategy. This is important because it allows the algorithm to focus on what would happen if you took a different action without considering all possible opponent strategies, which would be computationally expensive.

Now, for the example. Let's take a simplified poker hand. Suppose we have two players, Player 1 and Player 2. The game is limited to a single round of betting with a small number of possible bets. Each player has a hand, say Player 1 has a high card and Player 2 has a lower card. The game proceeds with Player 1 deciding to check or bet, then Player 2 responds, and so on.

In this scenario, the information sets for each player would be their possible hands and the actions taken so far. For Player 1, the information set might be just their own hand, and similarly for Player 2. The algorithm starts with uniform strategies, meaning each action has an equal probability. Then, in each iteration, it calculates the expected utility for each action and updates the regret accordingly.

Over time, the algorithm should converge to a Nash equilibrium where neither player can benefit by changing their strategy unilaterally. This is because each iteration minimizes the maximum regret, leading the strategies to stabilize.

However, I'm not entirely sure about the assumptions. I think one assumption is that the game is zero-sum, meaning one player's gain is the other's loss. Also, the game is deterministic, I believe. There might be an assumption about the game being perfect information, but I'm not certain. Another point is that CFR can handle imperfect information games, which is why it's used in poker.

As for limitations, I recall that CFR has a high computational complexity because it needs to traverse the entire game tree each iteration. For large games like full poker, this isn't feasible without further optimizations or approximations. Also, while it converges to a Nash equilibrium, it might take a lot of iterations, especially in complex games. Another limitation is that it's primarily designed for two-player games, and extending it to more players is challenging.

Wait, I'm a bit confused about the difference between counterfactual regret and regular regret. I think counterfactual regret considers the entire game's outcome as if the player had taken a different action at a specific point, without considering the prior actions. That's why it's called counterfactual—because it's a "what if" scenario.

So, putting it all together, the algorithm iteratively updates the strategies based on the regret values, ensuring that over time, the regret for any action diminishes, leading to a balanced strategy that's a Nash equilibrium.

I should also mention that in practice, CFR is often combined with other techniques to make it more efficient, like Monte Carlo sampling to approximate the game tree traversal instead of doing it exactly, which is necessary for large games.

I think I've covered the main points, but I need to make sure I explain each part clearly and provide a concrete example to illustrate how it works step by step.
</think>

**Counterfactual Regret Minimization (CFR) in Game-Theoretic Reinforcement Learning**

**Overview of CFR:**
CFR is an algorithm used in game theory to solve extensive-form games, particularly those with huge game trees like poker. It focuses on minimizing regret, which is the potential loss from not choosing a different strategy. The goal is to converge to a Nash equilibrium, where no player can benefit by unilaterally changing their strategy.

**Key Components and Process:**

1. **Regret and Strategy:**
   - Regret is the difference in utility had a different action been taken. CFR minimizes this regret over iterations.
   - Strategies are updated based on regret values, ensuring each action's probability reflects its desirability.

2. **Information Sets and Regret Tables:**
   - Extensive-form games are represented as trees with nodes as decision points. Information sets group nodes with identical histories.
   - Each information set maintains a regret table tracking the regret for each possible action.

3. **Iterative Process:**
   - The algorithm iterates, calculating strategies from current regrets and updating regrets based on counterfactual values.
   - Counterfactual values estimate the utility of taking an action assuming opponents follow their current strategy.

**Example: Simplified Poker Hand**

- **Game Setup:**
  - Two players, each with a specific hand (e.g., Player 1 has a high card, Player 2 a lower card).
  - Actions: check or bet, with possible responses.

- **Information Sets:**
  - Each player's set includes their hand and prior actions.

- **Algorithm Steps:**
  1. **Initialization:** Uniform strategies (equal action probabilities).
  2. **Iteration:** Compute expected utilities, update strategies, and adjust regrets.
  3. **Convergence:** Strategies stabilize as regret diminishes, reaching a Nash equilibrium.

**Assumptions and Limitations:**

- **Assumptions:**
  - Zero-sum games (one's gain is another's loss).
  - Deterministic outcomes, though extensions handle stochastic games.

- **Limitations:**
  - High computational complexity due to full game tree traversal.
  - Primarily for two-player games; extensions to more players are complex.
  - May require many iterations for convergence.

**Practical Enhancements:**
- **Monte Carlo Sampling:** Approximates game tree traversal for efficiency in large games.

**Conclusion:**
CFR effectively solves extensive-form games by iteratively minimizing regret, leading to a Nash equilibrium. While powerful, its computational demands and limitations in handling multi-player games highlight the need for further optimizations and research.

### DeepSeek API

In [32]:
# DeepSeek-R1-0528
llm_api = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-reasoner"

answer = get_output(llm_api, model_name, messages)

competitor_llms.append(model_name)
answers.append(answer)

display(Markdown(answer))

Of course. This is an excellent and deep question that gets to the heart of modern algorithmic game theory. Let's break down Counterfactual Regret Minimization (CFR) in detail.

### 1. The Core Concept: What is CFR?

**Counterfactual Regret Minimization (CFR)** is an iterative algorithm used to find an approximate **Nash Equilibrium** in two-player, zero-sum games with **imperfect information**, represented in their **extensive form** (i.e., as game trees).

The key intuition is this: Instead of solving the entire game at once, which is computationally intractable for large games like poker, CFR has each player learn to improve their strategy over time by measuring their **"regret."**

*   **Regret:** This is the difference between the utility (or payoff) a player *got* from using their current strategy and the utility they *would have gotten* if they had chosen a single, different action *every time they were in that same situation*. It answers the question: "How much do I wish I had done something else?"
*   **Counterfactual:** This term means "contrary to the facts." The algorithm analyzes hypothetical scenarios: "What would my payoff have been *if* I had chosen a different action, *given that* my opponent's strategy and my own strategy in all other parts of the game remained the same?" It's a localized thought experiment for each decision point.
*   **Minimization:** Over many iterations, the algorithm adjusts the player's strategy to minimize the overall (cumulative) regret. The celebrated **Regret Matching** algorithm is used for this update. If an action has positive regret (meaning we wish we had used it more), we increase the probability of choosing that action in the future.

The fundamental theorem behind CFR is that if both players' **average overall regret** is less than `ε` after `T` iterations, then their **average strategy** over those iterations is a `2ε`-Nash Equilibrium. As `T → ∞`, the average strategy converges to a Nash Equilibrium.

---

### 2. Algorithm Mechanics: Regret Tables and Information Sets

CFR operates by traversing the entire game tree on each iteration. The core components are:

#### **Information Sets (I)**
An information set is a group of game states that a player cannot distinguish between. For example, in poker, if your opponent bets, you don't know which exact cards they have; you only know the action they took. All states where the opponent has different cards but took the same action form a single information set for you.

#### **Regret Tables (Regret)**
For each information set `I` and for each possible action `a` at `I`, we store a numerical value `R(I, a)` representing the *cumulative regret* for not having taken action `a` in the past at this information set.

#### **Strategy Tables (Strategy)**
For each information set `I`, we maintain a probability distribution over actions, `σ(I, a)`, which defines the current strategy (the probability of taking action `a` at `I`).

#### **The CFR Algorithm (Simplified Overview)**

1.  **Initialize:** Set cumulative regrets and strategies to zero for all information sets and actions.
2.  **For each iteration `t`:**
    *   **Recursive Tree Traversal (CFR Function):** For a given player `i` and state `h`:
        *   **Base Case:** If `h` is a terminal state (e.g., a showdown in poker), return the payoff for player `i`.
        *   **Let the current player at `h` be `p`.**
        *   **For each action `a`:** Recursively call the CFR function on the next state `h · a` (state `h` followed by action `a`).
        *   **Calculate Node Utility:** The overall value `v` of the current state `h` is the weighted average of the values of the next states, weighted by the current strategy `σ(I, a)`.
        *   **Calculate Counterfactual Value:** For the acting player `p`, this is the value of the state `h` from their perspective, weighted by the probability that all other players (and chance) took to reach `h`. This "zooms in" on the player's contribution.
        *   **Calculate Immediate Regret:** For each action `a`, the regret is: `regret(a) = (value of taking action a) - (value of node h)`
        *   **Update Cumulative Regret:** Add this immediate regret (weighted by the counterfactual reach probability) to the cumulative regret for the information set `I` corresponding to `h`: `R(I, a) += regret(a)`
    *   **Update Strategy via Regret Matching:** After processing the entire tree, update the strategy for the next iteration for every information set `I`:
        *   `σ(I, a) = R⁺(I, a) / Σ_{b} R⁺(I, b)` where `R⁺` is the positive part of the regret (`max(0, R(I, a))`).
        *   If all cumulative regrets are non-positive, use a uniform random strategy.
3.  **Compute Average Strategy:** After many iterations, the **Nash Equilibrium strategy** is not the final strategy, but the *average strategy* across all iterations. This is computed by tracking the cumulative probability of reaching each information set and weighting the strategies used at each visit.

---

### 3. Concrete Example: Simplified Poker (Kuhn Poker)

**Game Rules:**
*   Two players, Ante of 1 chip each.
*   A deck with only three cards: Jack (J), Queen (Q), King (K). King beats Queen, Queen beats Jack, Jack beats King.
*   **Deal:** Each player gets one card, the third is unused.
*   **Betting:** One round of betting.
    *   Player 1 can either **Bet** (add 1 chip) or **Check**.
    *   If P1 checks, P2 can **Check** (showdown, higher card wins the pot) or **Bet**.
    *   If P1 bets, P2 can **Fold** (P1 wins the ante), or **Call** (add 1 chip, showdown for the larger pot).

**Let's simulate a few CFR iterations for a specific information set.**

**Scenario:** Player 2 is dealt the **Jack**. Player 1 **Bets**.
Player 2 doesn't know P1's card. So, from P2's perspective, P1 could have the Queen or the King. All game states where P2 has a Jack and P1 has bet form a single **information set**: `I = (P2 holds J, P1 has bet)`.

At this information set `I`, P2 has two actions: {**Fold**, **Call**}.



**Iteration 1:**
*   **Assume initial strategy is random:** `σ(Fold) = 0.5`, `σ(Call) = 0.5`.
*   **CFR traverses the tree for both possible cards P1 could have:**
    *   **Case 1: P1 has Q (beats J).** If P2 Calls, they lose 2 chips total (ante + call). If P2 Folds, they lose just 1 chip (the ante). The *value* of folding is better (-1 vs -2).
    *   **Case 2: P1 has K (loses to J).** If P2 Calls, they win 2 chips. If P2 Folds, they lose 1 chip. The *value* of calling is vastly better (+2 vs -1).
*   **Calculate Counterfactual Regret:**
    *   The "what if" value for Folding is the average of its outcomes: `(-1 + -1)/2 = -1`.
    *   The "what if" value for Calling is the average of its outcomes: `(-2 + 2)/2 = 0`.
    *   The overall value of the node (using current strategy) is `(0.5 * -1) + (0.5 * 0) = -0.5`.
    *   **Immediate Regret for Folding:** `-1 - (-0.5) = -0.5`
    *   **Immediate Regret for Calling:** `0 - (-0.5) = +0.5`
*   **Update Regret Table for `I`:** `R(Fold) += -0.5`, `R(Call) += +0.5`.

**Update Strategy for Iteration 2 (Regret Matching):**
*   Positive regrets: `R⁺(Fold)=0`, `R⁺(Call)=0.5`.
*   Total positive regret: `0 + 0.5 = 0.5`.
*   New strategy: `σ(Fold) = 0 / 0.5 = 0`, `σ(Call) = 0.5 / 0.5 = 1`.
*   P2's new strategy for this information set is to **Always Call**.

**Iteration 2:**
*   Now P2's strategy at `I` is to always Call (`σ(Call)=1`).
*   **CFR traverses the tree again:**
    *   The value of Calling is still `0`.
    *   The overall value of the node is now `(0 * -1) + (1 * 0) = 0`.
    *   **Regret for Folding:** `-1 - 0 = -1`
    *   **Regret for Calling:** `0 - 0 = 0`
*   **Update Regrets:** `R(Fold) = -0.5 + -1 = -1.5`, `R(Call) = 0.5 + 0 = 0.5`.
*   The strategy for iteration 3 remains `σ(Call)=1` because the positive regret for calling is still `0.5` and for folding is `0`.

**Convergence:**
Over thousands of iterations, this process continues. The algorithm will discover that while calling with a Jack is good against a bluffing King, it's terrible against a value-betting Queen. The regret for calling will be positive when P1 has a King and negative when P1 has a Queen. The cumulative regrets will balance out, and the **average strategy** will converge to a mixed strategy: for example, "Call 1/3 of the time and Fold 2/3 of the time when holding a Jack facing a bet." This mixed strategy makes P1 indifferent between betting with a Queen or checking, which is the hallmark of a Nash Equilibrium.

---

### 4. Assumptions and Limitations

1.  **Two-Player, Zero-Sum:** Vanilla CFR is designed for and guarantees convergence only in two-player, zero-sum games. Extensions to multiplayer and general-sum games exist (e.g., CFR-D) but are more complex and without the same convergence guarantees.
2.  **Finite Games:** The game must have a finite number of states and actions.
3.  **Perfect Recall:** Players must not forget information they once knew. (This is why information sets group states a player can't distinguish *at that moment*). CFR relies on this to structure the regret tables.
4.  **Computational Cost:** While more efficient than solving a Linear Program for the whole game, CFR still requires traversing the entire game tree on each iteration. For large games like full-scale poker (**~10¹⁶⁰** game states), this is impossible without significant sampling and abstraction techniques.
    *   **Monte Carlo CFR (MCCFR):** A common variant that only samples portions of the game tree on each iteration, drastically improving convergence speed in practice, though it remains computationally intensive.
5.  **Convergence Speed:** CFR converges to an `ε`-Nash Equilibrium in `O(1/ε²)` iterations, which can be slow for very small `ε`.
6.  **Memory Requirements:** Storing a regret value for every (information set, action) pair requires significant memory. For large games, this necessitates using **abstraction** (e.g., grouping similar hands in poker into "buckets") which introduces approximation error.

In summary, CFR is a groundbreaking algorithm that transformed AI for imperfect information games by framing the problem as one of regret minimization. Its success in creating superhuman poker AI like **Libratus** and **Pluribus** is a direct testament to its power, despite being bounded by the assumptions and limitations of the real world.

## Groups 3 LLMs - Open Source via Local Hosting (Ollama)

### Pull Models Locally

In [33]:
#Very small LLMs
!ollama pull llama3.2:1b # (https://ollama.com/library/llama3.2)
!ollama pull deepseek-r1:1.5b # (https://ollama.com/library/deepseek-r1)

#Small LLMs
!ollama pull llama3.2 #by default it's 3B (https://ollama.com/library/llama3.2)
!ollama pull phi3:mini   # (https://ollama.com/library/phi3)
!ollama pull gemma3:4b # ollama pull gemma3:4b-it-qat (https://ollama.com/library/gemma3)

#Slightly larger LLMs
!ollama pull qwen3:8b # (https://ollama.com/library/qwen3)
!ollama pull llama3.1 #by default it's 8B (https://ollama.com/library/llama3.1)
!ollama pull deepseek-r1:7b # (https://ollama.com/library/deepseek-r1)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 74701a8c35f6: 100% ▕██████████████████▏ 1.3 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 4f659a1e86d7: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling aabd4debf0c8: 100% ▕█████████

pulling 74701a8c35f6:  11% ▕██                ▏ 151 MB/1.3 GB   13 MB/s   1m26spulling manifest 
pulling 74701a8c35f6:  12% ▕██                ▏ 152 MB/1.3 GB   13 MB/s   1m26spulling manifest 
pulling 74701a8c35f6:  12% ▕██                ▏ 153 MB/1.3 GB   13 MB/s   1m26spulling manifest 
pulling 74701a8c35f6:  12% ▕██                ▏ 157 MB/1.3 GB   15 MB/s   1m14spulling manifest 
pulling 74701a8c35f6:  12% ▕██                ▏ 158 MB/1.3 GB   15 MB/s   1m14spulling manifest 
pulling 74701a8c35f6:  12% ▕██                ▏ 161 MB/1.3 GB   15 MB/s   1m14spulling manifest 
pulling 74701a8c35f6:  12% ▕██                ▏ 163 MB/1.3 GB   15 MB/s   1m14spulling manifest 
pulling 74701a8c35f6:  12% ▕██                ▏ 164 MB/1.3 GB   15 MB/s   1m14spulling manifest 
pulling 74701a8c35f6:  13% ▕██                ▏ 166 MB/1.3 GB   15 MB/s   1m14spulling manifest 
pulling 74701a8c35f6:  13% ▕██                ▏ 169 MB/1.3 GB   15 MB/s   1m14spulling manifest 
pulling 74701a8c35f6:  13% ▕██

pulling 74701a8c35f6:  29% ▕█████             ▏ 377 MB/1.3 GB   19 MB/s     48spulling manifest 
pulling 74701a8c35f6:  29% ▕█████             ▏ 379 MB/1.3 GB   19 MB/s     48spulling manifest 
pulling 74701a8c35f6:  29% ▕█████             ▏ 381 MB/1.3 GB   19 MB/s     48spulling manifest 
pulling 74701a8c35f6:  29% ▕█████             ▏ 383 MB/1.3 GB   19 MB/s     47spulling manifest 
pulling 74701a8c35f6:  29% ▕█████             ▏ 385 MB/1.3 GB   19 MB/s     47spulling manifest 
pulling 74701a8c35f6:  29% ▕█████             ▏ 387 MB/1.3 GB   19 MB/s     47spulling manifest 
pulling 74701a8c35f6:  29% ▕█████             ▏ 388 MB/1.3 GB   19 MB/s     47spulling manifest 
pulling 74701a8c35f6:  30% ▕█████             ▏ 390 MB/1.3 GB   19 MB/s     47spulling manifest 
pulling 74701a8c35f6:  30% ▕█████             ▏ 393 MB/1.3 GB   19 MB/s     47spulling manifest 
pulling 74701a8c35f6:  30% ▕█████             ▏ 394 MB/1.3 GB   19 MB/s     47spulling manifest 
pulling 74701a8c35f6:  30% ▕██

pulling 74701a8c35f6:  46% ▕████████          ▏ 604 MB/1.3 GB   19 MB/s     36spulling manifest 
pulling 74701a8c35f6:  46% ▕████████          ▏ 606 MB/1.3 GB   19 MB/s     36spulling manifest 
pulling 74701a8c35f6:  46% ▕████████          ▏ 607 MB/1.3 GB   19 MB/s     36spulling manifest 
pulling 74701a8c35f6:  46% ▕████████          ▏ 609 MB/1.3 GB   19 MB/s     36spulling manifest 
pulling 74701a8c35f6:  46% ▕████████          ▏ 612 MB/1.3 GB   19 MB/s     36spulling manifest 
pulling 74701a8c35f6:  46% ▕████████          ▏ 613 MB/1.3 GB   19 MB/s     36spulling manifest 
pulling 74701a8c35f6:  47% ▕████████          ▏ 615 MB/1.3 GB   19 MB/s     36spulling manifest 
pulling 74701a8c35f6:  47% ▕████████          ▏ 618 MB/1.3 GB   19 MB/s     35spulling manifest 
pulling 74701a8c35f6:  47% ▕████████          ▏ 619 MB/1.3 GB   19 MB/s     35spulling manifest 
pulling 74701a8c35f6:  47% ▕████████          ▏ 621 MB/1.3 GB   19 MB/s     35spulling manifest 
pulling 74701a8c35f6:  47% ▕██

pulling 74701a8c35f6:  63% ▕███████████       ▏ 830 MB/1.3 GB   19 MB/s     25spulling manifest 
pulling 74701a8c35f6:  63% ▕███████████       ▏ 831 MB/1.3 GB   19 MB/s     25spulling manifest 
pulling 74701a8c35f6:  63% ▕███████████       ▏ 834 MB/1.3 GB   19 MB/s     24spulling manifest 
pulling 74701a8c35f6:  63% ▕███████████       ▏ 836 MB/1.3 GB   19 MB/s     24spulling manifest 
pulling 74701a8c35f6:  63% ▕███████████       ▏ 837 MB/1.3 GB   19 MB/s     24spulling manifest 
pulling 74701a8c35f6:  64% ▕███████████       ▏ 839 MB/1.3 GB   19 MB/s     24spulling manifest 
pulling 74701a8c35f6:  64% ▕███████████       ▏ 842 MB/1.3 GB   19 MB/s     24spulling manifest 
pulling 74701a8c35f6:  64% ▕███████████       ▏ 843 MB/1.3 GB   19 MB/s     24spulling manifest 
pulling 74701a8c35f6:  64% ▕███████████       ▏ 845 MB/1.3 GB   19 MB/s     24spulling manifest 
pulling 74701a8c35f6:  64% ▕███████████       ▏ 847 MB/1.3 GB   19 MB/s     24spulling manifest 
pulling 74701a8c35f6:  64% ▕██

...

pulling 96c415656d37:  93% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     16spulling manifest 
pulling 96c415656d37:  93% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     16spulling manifest 
pulling 96c415656d37:  93% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     16spulling manifest 
pulling 96c415656d37:  93% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     16spulling manifest 
pulling 96c415656d37:  93% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     16spulling manifest 
pulling 96c415656d37:  93% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     15spulling manifest 
pulling 96c415656d37:  93% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     15spulling manifest 
pulling 96c415656d37:  93% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     15spulling manifest 
pulling 96c415656d37:  94% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     15spulling manifest 
pulling 96c415656d37:  94% ▕████████████████  ▏ 4.4 GB/4.7 GB   19 MB/s     15spulling manifest 
pulling 96c415656d37:  94% ▕██

pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕█████████████████ ▏ 4.6 GB/4.7 GB   19 MB/s      4spulling manifest 
pulling 96c415656d37:  98% ▕██

pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         pulling manifest 
pulling 96c415656d37: 100% ▕██████████████████▏ 4.7 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         pulling manifest 
pulling 96c415656d37: 100% ▕██████████████████▏ 4.7 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         pulling manifest 
pulling 96c415656d37: 100% ▕██████████████████▏ 4.7 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB               

pulling 40fb844194b2: 100% ▕██████████████████▏  487 B                         pulling manifest 
pulling 96c415656d37: 100% ▕██████████████████▏ 4.7 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         
pulling 40fb844194b2: 100% ▕██████████████████▏  487 B                         pulling manifest 
pulling 96c415656d37: 100% ▕██████████████████▏ 4.7 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         
pulling 40fb844194b2: 100% ▕██████████████████▏  487 B                         pulling manifest 
pulling 96c415656d37: 100% ▕██████████████████▏ 4.7 GB               

verifying sha256 digest ⠸ pulling manifest 
pulling 96c415656d37: 100% ▕██████████████████▏ 4.7 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         
pulling 40fb844194b2: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest ⠼ pulling manifest 
pulling 96c415656d37: 100% ▕██████████████████▏ 4.7 GB                         
pulling c5ad996bda6e: 100% ▕██████████████████▏  556 B                         
pulling 6e4c38e1172f: 100% ▕██████████████████▏ 1.1 KB                         
pulling f4d24e9138dd: 100% ▕██████████████████▏  148 B                         
pulling 40fb844194b2: 100% ▕██████████████████▏  487 B                         
verifying sha256 digest ⠴ pulling manifest 
pulling 96c415656d37: 100% ▕██████████████████▏ 4.7 GB              

In [34]:
#View list of downloaded models
!ollama ls

NAME                ID              SIZE      MODIFIED               
deepseek-r1:7b      755ced02ce7b    4.7 GB    Less than a second ago    
llama3.1:latest     46e0c10c039e    4.9 GB    1 second ago              
qwen3:8b            500a1f067a9f    5.2 GB    2 seconds ago             
gemma3:4b           a2af6cc3eb7f    3.3 GB    3 seconds ago             
phi3:mini           4f2222927938    2.2 GB    4 seconds ago             
llama3.2:latest     a80c4f17acd5    2.0 GB    5 seconds ago             
deepseek-r1:1.5b    e0979632db5a    1.1 GB    6 seconds ago             
llama3.2:1b         baf6a787fdff    1.3 GB    8 seconds ago             
llama3.1:8b         46e0c10c039e    4.9 GB    5 days ago                


### Getting Outputs

We will use the `OpenAI` Python client library to interact with our local Ollama instance running different models, by simply pointing the client to our Ollama server's URL (typically http://localhost:11434/v1).

In [35]:
# Ensure the model names match the pulled models in Ollama
model_names_ollama = [
    "llama3.2:1b",
    "deepseek-r1:1.5b",
    "llama3.2",
    "phi3:mini",
    "gemma3:4b",
    "qwen3:8b",
    "llama3.1",
    "deepseek-r1:7b"
]

In [46]:
for model_name in model_names_ollama:
    llm_api = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama') # note that llm_api is a good variable name, because the LLM is indeed exposed locally as an API by Ollama!

    answer = get_output(llm_api, model_name, messages)

    competitor_llms.append(model_name)
    answers.append(answer)

    display(model_name, Markdown(answer))

'llama3.2:1b'

Counterfactual Regret Minimization (CFR) is a game-theoretic reinforcement learning algorithm used to solve extensive-form games, particularly in situations where an agent wants to learn or play against another player or agent that may deviate from their optimal strategy. In a CFR agent, the goal is not to maximize predicted future rewards but to minimize regret over all possible deviant actions.

**Key Concepts:**

1. **Counterfactual regret**: A measure of how much worse a player would regret their actions if an adversary were to deviate from optimal strategy.
2. **Regret table**: A matrix that keeps track of the deviation risk for each state-action pair in the history, where the row represents the regret with respect to the current best response policy (i.e., playing against the top player), and the column represents all possible actions.

**Algorithm Overview:**

1. **Initialization**: Construct a regret table from scratch by initializing it as an empty matrix. The rows are used to represent the deviant action policies of both players, while the columns are used to represent all possible actions.
2. **Game Iteration**: Iterate over each state-action pair in the history using the Bellman-Gaussian function (BGF). For each tuple (ai,p) where ai is a particular player and p is an action:
    * Compute predicted rewards against both players' best responses for this action (R_pi|a) and against their actual strategies (R_pi).
    * Calculate counterfactual regrets by subtracting the maximum possible reward from R_pi.
    * Update the regret table with Bellman-Gaussian updating principle: \(\{r_{i}\}_{\delta,i}^\prime = r_i - q^T_q(a,\delta) - u(x_i, a|x_i; \sigma_i)\), where \(q^{T.q}(a,\delta) r\) is the predicted expected reward and \(u(x,a|x;\sigma)\) is the loss associated with playing against player i. The update rule updates the cell \(\{r_{i}\}_{\delta,i}^\prime\) if \(\{r_{i}\}_{\delta,i'} \leq {r_{i}}_{\delta,i}' - {r}_{i}\).
3. **Best Response Update**: Revisit all regret values in the regret table and choose the action that results in the highest counterfactual regret for each state-action pair, i.e., \(a^+ = argmax_r r_{i}^\prime - q(x_i) a \).

**How CFR Converges to a Nash Equilibrium:**

1. **Local Minimization**: In an extensive-form game with two distinct players, the CFR algorithm will converge locally to a Nash equilibrium because it ensures that both players optimize their strategy based on the expected rewards against each other's deviation (counterfactual regret).
2. **Global Convergence**: By iteratively refining the regret table, the agent can explore more states-action pairs and increase its chances of catching suboptimal deviations from other players' strategies.

**Assumptions and Limitations:**

1. **Homogeneous Environment**: CFR assumes that both players play with the same deviation risk model (BGF). If this is not possible in a real-world scenario, the algorithm might converge to an approximate Nash equilibrium based on observed deviations.
2. **Assuming No Feedback**: Since CFR relies on explicit information from other players' actions and deviation estimates, it requires feedback to know which deviant action was taken against whom.
3. **Lack of Action Space Compactness**: The regret table grows with the number of possible states-action pairs (state-action space). In games requiring exploration to converge like poker or other card games, CFR's potential computational limitations in handling dense state-action spaces need careful consideration.

**Example: Simplified Poker Hand**

Let's consider a simplified poker hand simulation for illustrating CFR's convergence to a Nash equilibrium. We have 4 players with the following deviation risk model assumptions (BGF):

- Deviation from top player = \(d_x\)
- Deviation from second player = \(-2d_2\)
- Deviation from three players = \(-3d_3\)

Initially, we initialize a regret table to zero.

### Sample Actions and Their Deviations:

| Player  | Action        | Deviation   |
|:--------|---------------|:-----------|
| P      | 1             | 0          |
| Q      | 2             | -10       |
| R      | -5            | 15         |
| T      | 6             | -20       |

### Iterative Update:

Using the BGF, update our counterfactual regrets as follows (using the above values):

- For player P:
  - \(m_1 = m_P\) (initial value)
  - Update \({r_{10}}\): Minimize \(\delta=2d_2 - (\max(0,\delta) / 10^{2}) + (\delta / 3^6)\cdot(-7d_3) + \delta / 4^{11} \le{u}_P(x) m_P - u_P(x, P/2+x) /8\) 
  - Update \({R_{PT}} = {r_T^\prime}\)
  
Similarly for the other players.

We start by setting \(m_1=m_T\), then proceed iteratively updating our regret values. By doing this, we find a Nash equilibrium where both players play their best responses to each other's deviation, assuming the deviations are homogeneous and no feedback mechanism. However, please note that the algorithm might fail if either player deviates disproportionately or does not have enough information to make informed decisions.

'deepseek-r1:1.5b'

<think>
Okay, so I need to explain what Counterfactual Regret Minimization (CFR) is, especially in the context of game-theoretic reinforcement learning for solving extensive-form games. Hmm, I remember learning about this in some AI and game theory classes, but it's a bit fuzzy. I'll try to piece together what I know.

First, what's an extensive-form game? Oh right, that's a sequential game where players make decisions at different points in the game, not just once. Each decision branches out into possible actions or states. So CFR is probably used on these kinds of games, especially for finding Nash Equilibria, which are stable situations where no player can benefit by changing their strategy while others keep theirs.

Now, what's a Regret Table? I think it's about updating strategies based on the information available at each decision point. In extensive-form games, you might have imperfect information, so not all moves are known to everyone initially. That means players could choose different branches of the game tree, and the regret table tracks how likely those branches were when specific actions were taken.

CFR stands for Counterfactual Regret Minimization. So it's about minimizing regret by considering alternative choices that didn't happen but would have if your initial assumption was wrong? Wait, no, counterfactual here means looking into what you'd have gained or lost if you had chosen another action when an option was taken.

So the algorithm works by traversing all possible game paths and updating each node with the corresponding regret. Each iteration of CFR considers a small subset of nodes that are active (i.e., have different information sets) and recalculates the regrets for those. Then, a new set of nodes is added to explore other possibilities.

Let me think about how this works step by step. Imagine a game where each decision point has multiple options. At one point, both players chose their first move without knowing what the opponent played before that. So, in counterfactual regret minimization, they look back and update their strategies based on how well those choices performed against alternative counterfactuals.

I should break it down: for each profile of pure strategies, perform multiple iterations where at each node, players evaluate the counterfactual outcomes if they had chosen different actions. Then, adjust their strategy to minimize the regret (or maximize expected utility).

Let me take a concrete example. Maybe poker isn't the best, but let's try something else. Let's say it's similar to tic-tac-toe in terms of sequential moves with imperfect information.

Wait, maybe I should pick a simpler game where I can visualize. Suppose a game where each move has two options, and after moving some nodes are pruned because they cannot be reached from later nodes. So for example, suppose the first move branches into A and B. If both players choose A, it leads to terminal outcomes. But if one chooses B, maybe there's another branch that is never possible or gets pruned.

So when computing the counterfactual regret tables, each time I'm considering the best response on an alternative branch at each node.

Another point: in practice, we don't need to track every possibility because pruned nodes aren't reached from future moves. So each iteration updates the states by adding new states that represent reaching other branches or removing others that are now impossible.

And under what conditions does this algorithm converge? In many cases, it seems to work, but sometimes there might be cycles or non-convergence if certain choices are not made correctly. But with pruning and focusing on relevant subgames, maybe it's effective enough for some games.

Lastly, how is this used in extensive-form games to find Nash equilibria? By iteratively refining the strategy distributions based on regret minimization at each decision point, I suppose this converges as if we're looking through all possible information sets and updating strategies where they can gain the highest expected utility.

Maybe I should also mention that it's computationally intensive because you have to consider a lot of nodes per iteration. But even so, in many practical applications, especially with perfect information games, it works well.

Wait, do I have any prior knowledge that conflicts this? For instance, I think CFR has been used to solve Leduc Texas Hold'em and other poker-like games effectively. So maybe the method is solid for these types of games where players' moves are known in some contexts but not others.

I should structure my answer by first explaining what CFR is, how it uses regret tables, walk through an example with pruning, discuss convergence, include a practical use case like simplified poker, and then talk about its limitations. Need to make sure I'm clear on all points without me being too vague.
</think>

**Counterfactual Regret Minimization (CFR) in Game-Theoretic Reinforcement Learning**

Complex games are often analyzed using game-theoretic reinforcement learning, particularly for extensive-form games where both players make sequential decisions with imperfect information. This method aims to find Nash Equilibria by iteratively refining strategies based on regret minimization. Here's a structured explanation:

### **What is CFR?**
Counterfactual Regret Minimization (CFR) is an algorithm designed to solve extensive-form games efficiently. It relies on iterated regressions over game trees, updating strategies incrementally by considering alternative choices at each node.

### **Regret Tables**
A Regret Table in CFR tracks the regret of not having chosen a particular branch of a game tree at a given information set. This table allows players to evaluate how their choices might have performed if they had deviated from their initial plans, aiming to minimize regret for themselves.

### **How CFR Constructs and Updates Regret Tables**
1. **Initial Setup**: Players traverse the extensive-formgame.
2. **Regret Calculation**: At each node, players assess counterfactual outcomes of alternative branches. The regret is the difference between what was gained or lost when a specific action was taken versus the best-performing alternative.
3. **Strategy Update**: Players adjust their strategies based on these regrets to minimize expected utility.
4. **Iterative Process**: Multiple iterations update each node and its neighboring information sets, incorporating new outcomes as paths are pruned.

### **Concrete Example: A Simplified扑克 Game**
Consider a simplified hand where choices branch into two options at an initial decision node (e.g., A or B). If A is chosen:
- It may lead to terminal outcomes.
- However, if B were chosen instead, there might be alternative paths that are pruned due to being unreachable.

In each iteration of CFR:
1. Analyze the Regret Table, focusing on the current active nodes.
2. For each node, calculate counterfactuals against unexplored alternative branches.
3. Update strategies to reflect minimal regret or maximal expected utility.
4. Add new nodes in pruned information sets and remove others.

### **Convergence**
While complex games might oscillate, CFR's pruning strategy iteratively refines the game tree, converging toward a Nash Equilibrium by considering all relevant game dynamics at each decision node.

### **Practical Use Case (Simplified Poker)**
Players update their strategies based on the regret table from each information set. This iterative process allows them to converge closer to optimal play, especially in perfect information games like Leduc Texas Hold'em, where convergence is effectively achieved through pruning and focused updates.

### **Limitations**
Despite its effectiveness, CFR's computational demands can be high due to extensive tree traversal and strategy updates. It works well for certain types of sequential games with imperfect information but may face challenges in complex or large-scale scenarios.

In essence, CFR iteratively minimizes regret by updating strategic choices across game nodes, making it a robust method for finding Nash Equilibria in structured games.

'llama3.2'

...

<think>
Okay, I need to explain counterfactual regret minimization (CFR) in game-theoretic reinforcement learning for solving extensive-form games. Let me start by recalling what I know about CFR. 

First, CFR is an algorithm used to find Nash equilibria in games, especially in zero-sum games like poker. It's part of the game theory and reinforcement learning intersection. The main idea is to minimize regret over time, which is the difference between the payoff from choosing a particular action and the best possible action in hindsight. 

Wait, regret in this context is about the counterfactual scenario. So, for each information set (which is a point in the game where a player has to make a decision), the algorithm tracks how much regret there is for not choosing each possible action. Over time, the algorithm adjusts the strategies to minimize this regret, leading towards an equilibrium where no player can benefit by unilaterally changing their strategy.

Now, how does CFR construct and update regret tables? Let me think. Each information set has a list of possible actions. For each action, the algorithm keeps track of the regret for not choosing that action. Initially, these regrets might be zero. Then, during the iteration, for each game tree, the algorithm plays the game, updates the regrets based on the outcomes, and then uses these regrets to adjust the strategy probabilities. 

The update process probably involves calculating the regret for each action at each information set and then normalizing the regret to get the new strategy. For example, if an action has high regret, it means that not choosing it led to worse outcomes, so the probability of choosing that action would increase in future iterations.

Let me try to outline the steps of CFR:
1. For each information set, maintain a regret table that records the cumulative regret for each action.
2. During each iteration, the algorithm plays the game, using current strategies to choose actions.
3. After each game, it updates the regret tables by considering the counterfactual scenario: what would have happened if the player had chosen a different action at that information set.
4. The regret is calculated as the difference between the payoff from the chosen action and the best possible payoff from any other action, adjusted for the subsequent moves.
5. The strategy for each information set is then updated by normalizing the regret table to form a probability distribution over actions.

Wait, but how exactly is the regret calculated? For a specific action at a particular information set, the regret would be the sum over all possible subsequent outcomes of the difference between the actual payoff and the payoff that would have occurred if the player had chosen a different action. But since the game is sequential, the regret is calculated over the entire game tree, considering all possible paths.

Let me think about the example. Let's take a simplified poker scenario. Suppose there's a game with two players, each has to decide whether to bet or fold. The game has a few information sets. For instance, the first player might have a hand that can be strong or weak. The second player, after seeing the first player's action, has to decide to call or fold.

In CFR, the algorithm would track the regret for each possible action at each information set. For example, at the first player's initial decision, the algorithm might have a strategy that's a mix of betting and folding. Then, during the game play, it updates the regret tables based on how each action performed. If betting led to higher payoffs in the counterfactual scenarios, the regret for not betting would be higher, leading the algorithm to adjust the strategy to increase the probability of betting.

But how does this process lead to convergence to a Nash equilibrium? Over time, the regret for each action should decrease, as the algorithm learns to choose actions that minimize the regret. Since in a Nash equilibrium, no player can improve their payoff by unilaterally deviating, CFR's minimization of regret should drive the strategies towards such a point.

However, I should mention the assumptions. For example, CFR assumes that the game is a zero-sum game, but can it handle non-zero-sum games? Wait, no. Wait, CFR is used for zero-sum games, but in general, it can be extended. Wait, no, the original CFR is for two-player zero-sum games. The convergence to Nash equilibrium is guaranteed under certain conditions, but for general games, it might require more steps.

Another assumption is that the game is an extensive form game with perfect recall. Perfect recall means that each player's strategy is consistent across information sets, so they remember all previous actions. CFR relies on this because it assumes that players don't lose track of their own actions.

Limitations: CFR can be computationally intensive, especially for games with many information sets. The complexity is exponential in the number of nodes, so for large games like poker, it's not feasible without some optimizations like using exploitability as a metric or using variants like CFR+ or average strategy iteration.

Now, trying to put this into a concrete example. Let's say a two-player poker game with a simple structure. Let's imagine a simplified version where Player 1 can bet or check, and Player 2 can call or fold. The game has three rounds, but maybe simplified to a single decision point for each player. Wait, maybe a better example is the "matching pennies" game, but that's a simultaneous move game. Alternatively, think of a game where each player has two actions at their own information sets.

Alternatively, consider a game where Player 1 has two cards, say high or low. Player 2 doesn't know the card. Player 1 can bet or check. Player 2 can call or fold. The payoffs depend on the cards. For example, if Player 1 has a high card and bets, Player 2 calls and gets a higher payoff. If Player 1 has a low card and checks, Player 2 might fold.

In CFR, for each information set (like Player 1's initial decision, Player 2's decision), the algorithm tracks regret. Initially, the strategies might be random, but over time, the regrets for each action are updated. For instance, if Player 1 bet and got a higher payoff, the regret for not betting (if they had a low card) would be calculated. Wait, but how does the regret calculation work in such a case?

Let me think of the steps. Suppose we're in the first iteration. Player 1 has a strategy that's 50% bet, 50% check. Player 2 has a similar strategy. They play the game, compute the outcomes, and then for each information set, compute the regret. 

For example, at Player 1's information set, suppose they bet. The regret for not betting would be the difference between the payoff from betting versus not betting. But since the game is sequential, the regret is calculated over the entire game. So, for each action, the algorithm considers the expected payoff if that action was taken, versus the actual payoff, and updates the regret accordingly.

But to get into the example, let's make it concrete. Let's say the game is as follows:

Player 1 has a card, which is either H or L (high or low) with equal probability. Player 1 can choose Bet or Check. If Player 1 bets, Player 2 can call or fold. If Player 2 calls, they split the pot (say, 1 unit each). If Player 1 checks, Player 2 can call or fold. However, the actual payoffs depend on the card. Suppose if Player 1 has a high card, betting leads to Player 2 calling, giving Player 1 a +1. If Player 1 has a low card, betting would lead to Player 2 folding, giving Player 1 a +1 (if they bet and win). Wait, maybe I need to adjust the payoffs. Let me think of a specific setup.

Alternatively, consider a simplified game where Player 1 has a high card, and the pot is 1. Player 1 can bet or check. If Player 1 checks, Player 2 can call or fold. If Player 2 calls, then the player with the higher card wins. If Player 1 has the high card, they win. If Player 1 checks and Player 2 folds, Player 1 takes the pot. But this might be too complex. Maybe a better example is a simple game with only two decision points and two players.

Alternatively, take the game of Matching Pennies but in an extensive form. However, that's a simultaneous game. Maybe the example is too vague. Let's try to simplify further.

Let me think of a simple game with two players, each with two information sets. Let's say the game has a root node where Player 1 chooses Action A or B, and then Player 2 chooses Action C or D. The payoffs are such that this is a zero-sum game. 

But to make the example concrete, maybe take a game where the players have to choose to raise or fold, with certain outcomes. For instance, in a two-player poker game where each player has one card, and the pot is determined based on their cards. The exact payoffs are such that the game has a unique Nash equilibrium.

In CFR, the algorithm would simulate multiple iterations, keeping track of the regret tables for each information set. Each time, it plays the game according to the current strategy, then updates the regret for each action. The regret for not taking a particular action is calculated by considering all the possible future paths and their expected payoffs.

After many iterations, the strategies converge to a Nash equilibrium because the regret for all actions diminishes. The average strategy across iterations approaches the equilibrium strategies. 

But I need to describe the example in detail. Let me think. Suppose Player 1 has a choice between Bet (B) or Fold (F) in the first round, and Player 2 has to respond in the second round. The exact game structure is a bit complex, but for simplicity, let's assume that:

- If Player 1 bets and Player 2 folds, Player 1 wins the pot (value 1).
- If Player 1 bets and Player 2 calls, then the winner is determined by the cards.
- If Player 1 folds, they lose immediately.

But for the example, maybe it's better to have a simple game where Player 1 has two cards, say, high and low, and the outcome is determined solely by the choices. Let me try to make it simple.

Let me consider a two-player zero-sum game with the following structure:

- Player 1 has two information sets: one when they have a high card (H), one when they have a low card (L). Each card has a 50% chance.
- At each information set, Player 1 can choose to Bet (B) or Check (C).
- If Player 1 checks, then Player 2 can choose to Call (C) or Fold (F).
- If Player 1 bets, Player 2 can Call (C) or Fold (F).
- The payoffs are as follows:
  - If Player 1 has H and bets, Player 2 folds: Player 1 gets +1, Player 2 gets -1.
  - If Player 1 has H and bets, Player 2 calls: Player 1 gets +1, Player 2 gets -1 (since Player 1's card is higher).
  - If Player 1 has L and bets, Player 2 folds: Player 1 gets +1, Player 2 gets -1 (since they are bluffing).
  - If Player 1 has L and bets, Player 2 calls: Player 1 gets -1 (because their card is lower), Player 2 gets +1.
  - If Player 1 checks, and Player 2 folds: Player 1 gets 0 (no pot).
  - If Player 1 checks, and Player 2 calls: Player 1 gets 1 if their card is high, else -1. Wait, maybe this needs to be adjusted.

Alternatively, maybe the payoffs are such that when Player 1 checks, the pot is 1, and if Player 2 calls, it's split. But I'm getting confused. Let me try to think again. For simplicity, let's define payoffs such that the game is a zero-sum game with the following payoffs:

- When Player 1 checks and Player 2 folds, Player 1 gets 1 (win).
- When Player 1 checks and Player 2 calls, it's a toss-up (but maybe 0 for both? Or depends on cards).
- But this is getting too convoluted. Maybe I can use a standard example like the Texas Hold'em game but simplified.

Alternatively, perhaps it's better to refer to a standard game used in CFR explanations. For example, in the game of "rock-paper-scissors" as an extensive form game, but that's a simultaneous game. Maybe the example is a game where each player has to decide their action in sequence, and the outcomes are straightforward.

Alternatively, let me refer to a game where Player 1 can choose to raise or fold in the first round, and Player 2 can call or fold in the second. Suppose the payoffs are:

- If Player 1 raises and Player 2 folds, Player 1 gets +1, Player 2 -1.
- If Player 1 raises and Player 2 calls, then Player 1 wins with probability p (e.g., p=0.5), Player 2 wins with probability 1-p.
- If Player 1 folds, Player 1 gets 0, Player 2 gets 0 (no pot).

But this is still a bit too vague. Let me think of a game with two information sets. Suppose Player 1 has the ability to raise or check, and Player 2 can call or fold. The payoffs are:

- If Player 1 raises and Player 2 folds: Player 1 gains 1, Player 2 loses 1.
- If Player 1 raises and Player 2 calls: Player 1 has a 50% chance of winning, Player 2 50%.
- If Player 1 checks and Player 2 folds: Player 1 gains 0.
- If Player 1 checks and Player 2 calls: Player 1 gains 0 (they both split the pot, or it's a tie?).

This might not be a perfect example, but it's simple enough to work through. Let me try to simulate how CFR would work in this scenario.

Initially, the regret tables for each information set (Player 1's raise/check and Player 2's respond) are zeros. Suppose during the first iteration, Player 1's strategy is 50% raise, 50% check. Player 2's strategy is 50% call, 50% fold. The game outcomes would vary, but for each game played, the algorithm computes the regret.

For example, if Player 1 raises, Player 2 folds: Player 1 gets +1. The regret for Player 1's not raising (if they had a different card) might be calculated. Wait, but in the CFR algorithm, the regret is calculated for each action at each information set, regardless of the state. Wait, no. Let me recall the CFR algorithm.

The CFR algorithm works by having each player maintain a regret table for each information set. For each information set, the algorithm maintains a vector of regrets for each action. Then, for each iteration, the algorithm plays the game using the current strategy (which is the average of the strategies from previous iterations), and after each game, it updates the regret tables.

The regret for an action at a particular information set is computed by comparing the payoff from choosing that action versus the best possible payoff from any other action, considering the subsequent game. However, since the game is extensive form, the calculation involves the expected payoffs of the subgame that follows.

The key idea is that for each action, the regret is accumulated over multiple rounds, and the strategy is updated to balance the regret, leading to a Nash equilibrium.

Now, in the example, suppose after multiple iterations, Player 1's strategy converges to raising with probability p and checking with probability 1-p, while Player 2's strategy is calling with probability q and folding with 1-q. The CFR algorithm ensures that the regret for each action in the information set is minimized, thus approaching the equilibrium.

Assumptions: The game must be a zero-sum game for CFR to find a Nash equilibrium (since in general, Nash equilibria in non-zero-sum games require more complex algorithms). Also, the game must have perfect recall, meaning players don't have to choose actions without knowing their previous moves.

Limitations: CFR can be computationally expensive for large games because it requires traversing the entire game tree for each iteration. As a result, it's often applied to games with a manageable size, and more efficient variants like CFR+ or using exploitability metrics are used for larger games.

In summary, the CFR algorithm uses regret minimization at each information set to approach a Nash equilibrium. The algorithm updates regret tables after each game, leading to strategies that minimize potential regrets over time, eventually converging to an equilibrium.
</think>

**Counterfactual Regret Minimization (CFR)** is a foundational algorithm in game-theoretic reinforcement learning, used to solve **extensive-form games (EFGs)**, particularly zero-sum games like poker. CFR leverages **regret minimization** to iteratively refine strategies, aiming to converge toward a **Nash equilibrium**, where no player can benefit by unilaterally deviating from their strategy.

---

### **Key Concepts of CFR**
1. **Regret and Counterfactual Regret**:
   - **Regret** measures the loss incurred by not choosing a specific action at a particular decision point (information set).
   - **Counterfactual Regret** extends this by considering the hypothetical outcomes if the player had chosen a different action, **accounting for the entire game tree** that follows.

2. **Information Sets and Regret Tables**:
   - Each **information set** (a node where a player must act, given their knowledge of previous moves) has a **regret table** that tracks the cumulative regret for each possible action.
   - CFR updates these tables by comparing the actual payoff of the chosen action against the **best possible payoff** from any alternative action in the subgame that follows.

3. **Strategy Update**:
   - After updating regret tables, CFR normalizes them to compute **strategy probabilities**—that is, the probability of choosing an action at an information set. Actions with higher regret are prioritized in subsequent iterations.

---

### **Algorithm Construction and Updates**
1. **Initialization**:
   - For each information set, initialize a regret table with zeros for all actions.

2. **Iterative Play**:
   - For each iteration (often called a **"round"**):
     - **Simulate the game** using the current strategies (initially random or uniform).
     - **Compute payoffs** for each player at terminal nodes.
     - **Backtrack through the game tree** to compute **counterfactual regrets**:
       - For each action at an information set, the regret is the difference between the actual payoff and the expected payoff of the best alternative action (considering the subgame).
     - **Update the regret tables** by adding the computed regret to their respective entries.

3. **Strategy Refinement**:
   - After each round, **normalize the regret tables** to derive the **current strategy** for each information set. The strategy is the **softmax of the regret vector**, allocating higher probabilities to actions with lower (or negative) regret.

4. **Convergence**:
   - Over iterations, the regret for all actions at all information sets converges to **zero**, indicating that the strategy is a **Nash equilibrium**. The **average strategy across iterations** approximates the equilibrium.

---

### **Example: Simplified Poker Game**
Consider a game where:
- **Player 1** (the raiser) has two cards: **High (H)** or **Low (L)**, each with 50% probability.
- **Player 1** can **Raise (R)** or **Check (C)**.
- **Player 2** (the responder) can **Call (C)** or **Fold (F)**.
- **Payoffs**:
  - If Player 1 **Raises** and Player 2 **Folds**: Player 1 wins +1, Player 2 loses -1.
  - If Player 1 **Raises** and Player 2 **Calls**: Player 1 wins with 50% chance, Player 2 wins with 50%.
  - If Player 1 **Checks** and Player 2 **Folds**: Player 1 gets 0.
  - If Player 1 **Checks** and Player 2 **Calls**: Player 1 wins with 50% chance if they have H, else loses.

#### **CFR Execution**:
1. **Initialization**: Regret tables for Player 1's Raise/Check and Player 2's Call/Fold are initialized to zeros.
2. **Iteration 1**:
   - Player 1 randomly raises or checks (e.g., 50% each).
   - Player 2 randomly calls or folds (50% each).
   - For each game outcome, compute counterfactual regrets (e.g., Player 1 gains regret for not checking if they had H).
   - Update regret tables accordingly.
3. **Subsequent Iterations**:
   - Strategies are refined to minimize regret. For example, Player 1 may raise more often when they have H, and Player 2 might call more aggressively when facing a raise.
4. **Convergence**:
   - After many iterations, strategies stabilize, and the average strategy across all rounds approaches an equilibrium where both players cannot improve their expected payoff by unilaterally changing their actions.

---

### **Key Assumptions and Limitations**
- **Zero-Sum Assumption**: CFR guarantees a Nash equilibrium only in **zero-sum games** (e.g., two-player games). For non-zero-sum games, additional mechanisms are required.
- **Perfect Recall**: The game must have **perfect recall** (players cannot forget their history of actions).
- **Computational Complexity**: CFR has **exponential time complexity** in the number of nodes, making it suitable for small games. Larger games require **variants** like **CFR+** or **Exploitability-Guided Search**.

---

### **Conclusion**
CFR is a powerful framework for solving extensive-form games by iteratively minimizing regret. Through its systematic updates of regret tables and strategy probabilities, CFR ensures players converge to equilibrium strategies, even in complex scenarios like poker. Its simplicity and theoretical guarantees make it a cornerstone of game-theoretic AI, though practical applications often require optimizations for scalability.

'llama3.1'

Counterfactual Regret Minimization (CFR) is a popular algorithm used in game-theoretic reinforcement learning to solve extensive-form games, such as poker and Texas Hold'em. The algorithm was introduced by John Van de Walle et al. in 2009.

**Concept of CFR**

CFR is based on the idea of regret minimization, which involves computing the average regret (i.e., the loss incurred by not choosing an action) for each possible action-choice at information sets. An information set is a node in the game tree where the player has incomplete information about their opponent's actions. The algorithm constructs a regret table at each information set to store these calculated regrets.

The key insights behind CFR are:

1. **Regret minimization**: By minimizing the average regret, CFR aims to find an optimal policy that maximizes rewards or reduces losses.
2. **Counterfactual thinking**: At each information set, CFR considers not only the actual history but also all possible histories (including those where a different action might be taken).

**Algorithmic steps**

Given a game with N players and M actions at each player's turn, the CFR algorithm can be described as follows:

1. **Initialization**: Initialize all regret values to 0 in the regret tables.
2. **Simulation loop**:
	* For each player in the game tree, starting from any root node (i.e., an initial node where no information is known):
		+ Construct the game tree by simulating a random sequence of moves for each player up to an information set.
		+ Compute the regret at that information set assuming all possible histories have occurred with equal probability.
3. **Regret updates**:
	* Update the regret tables using the calculated regrets from the previous simulation loop.

The algorithm repeats this process of construction and update until a stopping criterion is met (e.g., a predefined number of iterations or convergence).

**Concrete example: Simplified Poker Hand**

Let's consider a 4-player, 5-round simplified poker game. We describe it here, but the specifics will be omitted for brevity.

### Problem formulation

*   Four players with equal chances to act at each node
*   3 information sets per player
*   A maximum of two actions available at each information set (e.g., bet or check)

This example is used throughout this explanation.

'deepseek-r1:7b'

<think>
Okay, so I'm trying to understand what Counterfactual Regret Minimization (CFR) is, especially as it's used in game-theoretic reinforcement learning for extensive-form games. It sounds like it's an algorithm that helps agents learn optimal strategies through repeated play. But honestly, the term "counterfactual" makes me a bit nervous because I'm more familiar with counterfactuals being related to causality and hypothetical scenarios—maybe that plays into how CFR works.

The user mentioned that CFR constructs and updates regret tables across information sets. Hmm, so extensive-form games have these decision points (nodes) where players make choices based on their information at different times. Information sets group nodes where a player can't distinguish between them because they don't have enough information to make a unique choice. So in each information set, the algorithm must somehow track how satisfied or regretful the current strategy is compared to other possible strategies.

I should probably break this down into parts: first explaining what CFR is, then how it works step by step (especially about the regret tables), and finally maybe walk through an example using a simplified poker hand. Let's see...

CFR allows agents to learn Nash equilibria in games where the agent can't model the opponent perfectly because those models are too big or complex for computational constraints. So, instead of perfect play against known opponents, it helps find a balanced strategy that no player wants to deviate from.

The algorithm works by iterating through each round, making decisions at nodes based on current strategies, recording outcomes, and then updating regrets when better options were available but not taken, considering both the actual events and counterfactual scenarios where different choices might have been made.

In code terms, CFR involves maintaining a strategy (like a behavior policy) for all decision points in each round. Then, it uses this to play out games against the current opponent model, which is essentially another instance of the algorithm itself. For every action taken at an information set during play, it calculates how much regret was felt when considering alternative strategies, both for that exact scenario and any possible alternatives that could have been chosen instead.

In terms of updating these regret values in a tabular way (like maintaining tables keyed by each node), I think the algorithm accumulates the difference between what would have been gained if an alternative strategy had been used versus what was actually gained. Over time, as play progresses through many rounds, strategies that result in lower regret should converge toward Nash equilibrium.

Taking the two-round poker example with imperfect information for one card could make sense here. Each player at a decision point (like deciding to go all-in or fold) has choices that lead to different payoffs depending on what's actually revealed later (the opponent's card). Using CFR, each time the game is played out in simulation by a copy of itself, it updates its strategies based on which actions would have provided better outcomes if taken.

So after each round, when all game instances are processed and the play sequences and payoffs are known, the algorithm goes through each information set (nodes) in reverse order to compute the regrets. For each node (like being dealt a specific card or making an action), it considers what would have happened had the player chosen another strategy but adjusted for both their actual gains and hypothetical counterfactual outcomes where alternate actions could have been taken.

The key is that over time, these regret values inform how often each decision point should be played based on past performances. So in the two-round poker example, after many iterations, the strategies (how often to go all-in or fold) would balance out such that neither player can gain by unilaterally changing their strategy—that's the Nash equilibrium.

I'm a bit concerned about potential limitations here, though. For one, computing exactly how the regret is updated across information sets might be computationally intensive for complex games because every node has to go through this process each iteration. Also, I wonder if there are cases where CFR converges too slowly or doesn't find the true Nash equilibrium.

In terms of assumptions, maybe it's assuming that all players follow deterministic best-response strategies given their current models, without considering that opponents might adapt adversarially. Additionally, in extensive-form games with perfect recall (like poker), the behavior policies can be updated correctly because they account for past observations accurately.

Overall, CFR is a way to iteratively improve each player's strategy by learning from past regrets, both real and hypothetical counterfactual scenarios. It’s particularly useful when models are too large to capture exact play-by-play opponent strategies due to computational constraints.
</think>

Counterfactual Regret Minimization (CFR) is an iterative algorithm used in game-theoretic reinforcement learning for extensive-form games. It helps agents learn optimal strategies, often converging towards a Nash equilibrium through repeated play.

### Explanation of CFR:
1. **Iterative Strategy Improvement**: CFR updates each player's strategy based on the regret associated with not having chosen alternative actions at decision points.
2. **Counterfactual Scenarios**: The algorithm evaluates both actual outcomes and hypothetical (counterfactual) scenarios where different choices might have been made, even if not taken in practice.
3. **Regret Calculation**: Regrets are maintained for each node in the game tree across information sets. Regret represents the difference in payoff between what was actually received and what would have been received with a different strategy.

### How CFR Updates Regret Tables:
- Each round involves playing out the game through a policy (strategy) for all decision points.
- For every action taken at an information set, the algorithm tracks how much regret is felt by considering alternative strategies. This includes both actual payoffs and counterfactual outcomes from different actions available to the player.

### Example Walkthrough: Two-Round Simplified Poker
1. **Game Setup**: Player 1 receives a card and has the option to go all-in or fold, while Player 2 also decides based on their card.
2. **Information Sets**: Nodes grouped by incomplete information (e.g., both players know neither's card but can act before it is revealed).
3. **Process**:
   - In each round, the game is simulated through a copy of itself with current strategies.
   - After processing all plays from a simulation, play proceeds back to update regrets at each node in reverse order using postdecision utilities.

### Convergence to Nash Equilibrium:
- Over many iterations (rounds), the algorithm's strategies adjust based on past regrets, leading towards equilibrium where no player benefits by unilaterally changing their strategy.
- In poker, this results in balanced strategies for both players, ensuring neither can gain an advantage by altering their actions.

### Limitations and Assumptions:
- **Computational Intensity**: CFR's computational cost grows with the game's complexity as it involves updating regrets across many nodes each iteration.
- **No Adversarial Adaptation**: It assumes opponents follow deterministic best-response strategies without adversarial adaptation, though this can be addressed in variants like FCFR (Fictitious Play + CFR) for more complex scenarios.

### Conclusion:
CFR efficiently iteratively improves strategies by learning from both real and counterfactual outcomes, making it suitable for large-scale extensive-form games where exact modeling of opponents is impractical. Despite potential computational challenges, the algorithm provides a practical approach to approaching Nash equilibrium in practice.

## Judgment Time

In [47]:
#Raw way to see resutls
print(competitor_llms)
print(answers)

['gpt-4.1-mini', 'gpt-4o-mini', 'claude-3-7-sonnet-latest', 'gemini-2.0-flash', 'llama-3.3-70b-versatile', 'deepseek-r1-distill-llama-70b', 'deepseek-reasoner', 'llama3.2:1b', 'deepseek-r1:1.5b', 'llama3.2', 'phi3:mini', 'gemma3:4b', 'qwen3:8b', 'llama3.1', 'deepseek-r1:7b', 'llama3.2:1b', 'deepseek-r1:1.5b', 'llama3.2', 'phi3:mini', 'gemma3:4b', 'qwen3:8b', 'llama3.1', 'deepseek-r1:7b']
['Certainly! Below is a detailed explanation of **Counterfactual Regret Minimization (CFR)** as used in game-theoretic reinforcement learning, specifically for solving **extensive-form games** such as poker.\n\n---\n\n### Overview: What is CFR?\n\n**Counterfactual Regret Minimization (CFR)** is an iterative algorithm designed to find approximate **Nash equilibria** in extensive-form games (games with sequential moves, imperfect information, and chance events). It was introduced by Zinkevich et al. (2007) and has been very influential in computational poker and other domains involving imperfect informat

In [48]:
#Using zip to iterate over both lists at once 
for competitor, answer in zip(competitor_llms, answers):
    print(f"Competitor: {competitor}\n\n{answer}")

Competitor: gpt-4.1-mini

Certainly! Below is a detailed explanation of **Counterfactual Regret Minimization (CFR)** as used in game-theoretic reinforcement learning, specifically for solving **extensive-form games** such as poker.

---

### Overview: What is CFR?

**Counterfactual Regret Minimization (CFR)** is an iterative algorithm designed to find approximate **Nash equilibria** in extensive-form games (games with sequential moves, imperfect information, and chance events). It was introduced by Zinkevich et al. (2007) and has been very influential in computational poker and other domains involving imperfect information.

**Key idea:**  
CFR minimizes **counterfactual regret** for each player over possible actions at each information set. Over many iterations, the average strategy converges to a Nash equilibrium strategy.

---

### Key Concepts

1. **Extensive-form games**:

   - Represented as a game tree.
   - Players do not have perfect information—nodes are grouped into **inform

In [49]:
# Using enumerate to iterate over the list and get the index
together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"
print(together)

# Response from competitor 1

Certainly! Below is a detailed explanation of **Counterfactual Regret Minimization (CFR)** as used in game-theoretic reinforcement learning, specifically for solving **extensive-form games** such as poker.

---

### Overview: What is CFR?

**Counterfactual Regret Minimization (CFR)** is an iterative algorithm designed to find approximate **Nash equilibria** in extensive-form games (games with sequential moves, imperfect information, and chance events). It was introduced by Zinkevich et al. (2007) and has been very influential in computational poker and other domains involving imperfect information.

**Key idea:**  
CFR minimizes **counterfactual regret** for each player over possible actions at each information set. Over many iterations, the average strategy converges to a Nash equilibrium strategy.

---

### Key Concepts

1. **Extensive-form games**:

   - Represented as a game tree.
   - Players do not have perfect information—nodes are grouped into **in

In [50]:
# Tell the judge LLM the task, with a prompt
judge = f"""You are judging a competition between {len(competitor_llms)} competitors.
Each model has been given these questions:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""

print(judge)

You are judging a competition between 23 competitors.
Each model has been given these questions:

Explain the concept of “counterfactual regret minimization” (CFR) as used in game‑theoretic reinforcement learning for solving extensive‑form games, describe how the algorithm constructs and updates regret tables across information sets, and illustrate with a concrete example (e.g., a simplified poker hand) how CFR converges to a Nash equilibrium, including a discussion of any assumptions or limitations of the method.

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}

Here are the responses from each competitor:

# Response from competitor 1

Certainly! Below is a detailed explanation of **Counterfactual Regret Minimization (CFR)** as used in game-theoret

In [51]:
judge_messages = [{"role": "user", "content": judge}]

In [52]:
# Get the results from the judge
llm_api = OpenAI()
model_name = "gpt-4.1-mini"

results = get_output(llm_api, model_name, judge_messages)

print(results)

{"results": ["1", "7", "4", "3", "21", "15", "11", "12", "19", "23", "2", "5", "6", "17", "9", "20", "16", "13", "14", "18", "8", "22"]}


In [53]:
#Turn this into results
results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitor_llms[int(result)-1]
    if competitor not in model_names_ollama + ["deepseek-chat","deepseek-reasoner", "llama-3.3-70b-versatile","deepseek-r1-distill-llama-70b"]:
        print(f"Rank {index+1}: {competitor} (Closed-source)")
    else:
        print(f"Rank {index+1}: {competitor} (Open-source)")

Rank 1: gpt-4.1-mini (Closed-source)
Rank 2: deepseek-reasoner (Open-source)
Rank 3: gemini-2.0-flash (Closed-source)
Rank 4: claude-3-7-sonnet-latest (Closed-source)
Rank 5: qwen3:8b (Open-source)
Rank 6: deepseek-r1:7b (Open-source)
Rank 7: phi3:mini (Open-source)
Rank 8: gemma3:4b (Open-source)
Rank 9: phi3:mini (Open-source)
Rank 10: deepseek-r1:7b (Open-source)
Rank 11: gpt-4o-mini (Closed-source)
Rank 12: llama-3.3-70b-versatile (Open-source)
Rank 13: deepseek-r1-distill-llama-70b (Open-source)
Rank 14: deepseek-r1:1.5b (Open-source)
Rank 15: deepseek-r1:1.5b (Open-source)
Rank 16: gemma3:4b (Open-source)
Rank 17: llama3.2:1b (Open-source)
Rank 18: qwen3:8b (Open-source)
Rank 19: llama3.1 (Open-source)
Rank 20: llama3.2 (Open-source)
Rank 21: llama3.2:1b (Open-source)
Rank 22: llama3.1 (Open-source)


In [54]:
# Get the results from another judge
llm_api = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

results = get_output(llm_api, model_name, judge_messages)

print(results)

{"results": ["4", "7", "1", "12", "13", "20", "21", "17", "15", "3", "2", "23", "19", "6", "11", "14", "10", "16", "5", "18", "8", "22", "9"]}


In [55]:
#Turn this into results
results_dict = json.loads(results.split("\n")[1])
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitor_llms[int(result)-1]
    if competitor not in model_names_ollama + ["deepseek-chat","deepseek-reasoner", "llama-3.3-70b-versatile","deepseek-r1-distill-llama-70b"]:
        print(f"Rank {index+1}: {competitor} (Closed-source)")
    else:
        print(f"Rank {index+1}: {competitor} (Open-source)")

IndexError: list index out of range

You can run the evaluations using multiple judges and take an ensemble approach to determine the best model! On another dimension, you can also increase the number of questions asked to the LLMs and include that as well resulting in a holistic evaluation!

# How Using AI Gateway Streamlines LLM usage

## Getting API Keys Ready AND Initializing 1 Common LLM API

In [ ]:
# Only 1 API Key needed!
tfy_api_key = os.environ["TFY_API_KEY"]

In [ ]:
#1 Common API
llm_api_tfy = OpenAI(api_key=tfy_api_key, base_url="https://internal.devtest.truefoundry.tech/api/llm")

## Initializing Task (comparing performance across LLMs)

In [ ]:
# Wrapping Steps 1-3 in a function AND modify a bit
def perform_analysis(llm_api, model_name, messages):
    response = llm_api.chat.completions.create(
        model=model_name,
        messages=messages
)
    answer = response.choices[0].message.content
    competitor_llms.append(model_name)
    answers.append(answer)

    display(Markdown(answer))

In [ ]:
# question = "question ......"

In [ ]:
competitor_llms = [] #list of LLMs that'll be called
answers = [] #answers from each LLM
messages = [{"role": "user", "content": question}] #messages

## Groups 1 LLMs - Closed Source

### OpenAI API

In [ ]:
# GPT 4.1-Mini
model_name = "llm-gateway-test-openai/gpt-4-1-mini" #from AI Gateway

In [ ]:
# perform_analysis(llm_api_tfy, model_name, messages)

In [ ]:
# GPT 4o-Mini
model_name = "llm-gateway-test-openai/gpt-4o-mini" #from AI Gateway

In [ ]:
perform_analysis(llm_api_tfy, model_name, messages)

### Anthropic API

**Note that Anthropic has a slightly different API, and max tokens is also required (not OpenAI compatible!) but it's streamlined with AI Gateway**

In [ ]:
# Claude 3.7 Sonnet
model_name = "llm-gateway-test-anthropic/claude-3-7-sonnet-latest" #from AI Gateway

In [ ]:
perform_analysis(llm_api_tfy, model_name, messages)

### Google API

In [ ]:
# Gemini 2.0 Flash
model_name = "llm-gateway-test-gemini/gemini-2-0-flash" #from AI Gateway

In [ ]:
perform_analysis(llm_api_tfy, model_name, messages)

## Groups 2 LLMs - Open Source via API

### Groq API

In [ ]:
# Llama 3.3 70B through Groq
model_name = "test-groq-prod-key/llama-3-3-70b-versatile" #from AI Gateway

In [ ]:
perform_analysis(llm_api_tfy, model_name, messages)

In [ ]:
# DeepSeek-R1-Distill-Llama-70B through Groq
model_name = "test-groq-prod-key/deepseek-r1-distill-llama-70b" #from AI Gateway

In [ ]:
perform_analysis(llm_api_tfy, model_name, messages)

### DeepSeek API - Currently not supported

## Groups 3 LLMs - Open Source via Hosting on TrueFoundry (Ollama)

### Pull Models (after Service Deployment)

In [ ]:
# Ollama account creation - WIP